In [1]:
%pip install langgraph langchain langchain-groq typing_extensions langchain-core ipython langchain-community
%pip install langchain-faiss
%pip install langchain-google-genai
%pip install pygraphviz
%pip install beautifulsoup4


from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage, RemoveMessage, AIMessageChunk
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.graph.message import MessagesState, add_messages
from langgraph.checkpoint.memory import MemorySaver

from IPython.display import Image, display
from typing_extensions import TypedDict
from typing import Literal

from langgraph.types import interrupt, Command
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pygraphviz-1.14.tar.gz (106 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pygraphviz
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [88 lines of output]
      C:\Users\Ashish\AppData\Local\Temp\pip-build-env-68mht6r6\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              *********************************************

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain.vectorstores import FAISS
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader

import os
os.environ["GOOGLE_API_KEY"] = ""


llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash',temperature=0,api_key="")
embedding_model = GoogleGenerativeAIEmbeddings(model = "models/text-embedding-004")

In [3]:
%pip install pymupdf
loader = PyMuPDFLoader("class10phy.pdf")
docs = loader.load()


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [135]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 100)
split_docs = splitter.split_documents(docs)

In [136]:
%pip install faiss-cpu

vector_store = FAISS.from_documents(embedding = embedding_model, documents = split_docs)
vector_store.save_local(folder_path='/content', index_name='class_10_phy')

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [137]:
vector_store = FAISS.load_local(folder_path='/content', index_name='class_10_phy', embeddings = embedding_model, allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever(search_type = "mmr", search_kwargs = {"k" : 5})

In [138]:
query = "what are the 2 laws of reflection?"
relevant_docs = retriever.invoke(query)
relevant_docs = [m.page_content for m in relevant_docs]

In [139]:
prompt = """You are given the query from the user , along with the content extracted form the vector embeddings related to the query.
Using this give the answer to the query.
These are questions related to a physics chapter of class 10.
Give the response of the solution in a complete way.
User Query: {query}
Relevant Content: {relevant_docs}
If the given content doesn't include the answer to the query, Give the response as 'I don't know'
"""
new_prompt = ChatPromptTemplate.from_template(prompt)

chain = new_prompt | llm
response = chain.invoke({"query": query, "relevant_docs" : relevant_docs})
print(response.content)


The two laws of reflection are:

1.  **The angle of incidence is equal to the angle of reflection.**
2.  **The incident ray, the normal to the mirror at the point of incidence, and the reflected ray, all lie in the same plane.**

These laws are applicable to all types of reflecting surfaces, including spherical surfaces.


In [140]:
import uuid
import json
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load PDF using PyMuPDF
loader = PyMuPDFLoader("class10phy.pdf")
docs = loader.load()

# Split documents with overlap
splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
split_docs = splitter.split_documents(docs)

# Metadata and UUID assignment
metadata_json = []

for i, doc in enumerate(split_docs):
    page = doc.metadata.get("page", 0) + 1  # PyMuPDF is 0-indexed
    uid = str(uuid.uuid4())
    chunk_text = doc.page_content.strip()

    # Update LangChain doc metadata (for potential future use)
    doc.metadata.update({
        "uuid": uid,
        "page": page,
        "source": "class10phy.pdf"
    })

    # Store in metadata JSON
    metadata_json.append({
        "uuid": uid,
        "page": page,
        "chunk_id": f"page_{page}_chunk_{i}",
        "text": chunk_text,
        "source": "class10phy.pdf"
    })

# Save metadata to file
with open("chunk_metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata_json, f, indent=2, ensure_ascii=False)


In [141]:
import json
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document

# --- Load vector store ---
vector_store = FAISS.load_local(
    folder_path='/content',
    index_name='class_10_phy',
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})

# --- Load chunk metadata JSON ---
with open("chunk_metadata.json", "r", encoding="utf-8") as f:
    chunk_metadata = json.load(f)
metadata_lookup = {entry["text"]: entry for entry in chunk_metadata}

# --- Run the query ---
query = "what are the 2 laws of reflection?"
docs: list[Document] = retriever.invoke(query)
relevant_chunks = [doc.page_content for doc in docs]

# --- Prepare prompt and invoke LLM ---
prompt = """You are given the query from the user, along with the content extracted from the vector embeddings related to the query.
Using this, give the answer to the query.
These are questions related to a physics chapter of class 10.
Give the response in a complete sentence.
User Query: {query}
Relevant Content: {relevant_docs}
If the given content doesn't include the answer to the query, respond with 'I don't know'."""

new_prompt = ChatPromptTemplate.from_template(prompt)
chain = new_prompt | llm
response = chain.invoke({"query": query, "relevant_docs": relevant_chunks})

# --- Print final answer ---
print("✅ Answer:", response.content)
print("\n🔍 Chunks used:\n" + "-" * 60)

# --- Print metadata and chunk content ---
for chunk_text in relevant_chunks:
    match = metadata_lookup.get(chunk_text)
    if match:
        print(f"🆔 Chunk ID: {match['chunk_id']}")
        print(f"📄 Page: {match['page']}")
        print(f"📄 Text: {match['text']}\n{'-' * 60}")


✅ Answer: The two laws of reflection are: (i) the angle of incidence is equal to the angle of reflection, and (ii) the incident ray, the normal to the mirror at the point of incidence, and the reflected ray, all lie in the same plane.

🔍 Chunks used:
------------------------------------------------------------


In [142]:
import uuid
import json
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document

# --- Load PDF and split into chunks ---
loader = PyMuPDFLoader("class10phy.pdf")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(docs)

# --- Assign UUIDs + metadata + save JSON ---
metadata_json = []
for i, doc in enumerate(split_docs):
    page = doc.metadata.get("page", 0) + 1
    uid = str(uuid.uuid4())
    chunk_text = doc.page_content.strip()

    doc.metadata.update({
        "uuid": uid,
        "page": page,
        "source": "class10phy.pdf"
    })

    metadata_json.append({
        "uuid": uid,
        "page": page,
        "chunk_id": f"page_{page}_chunk_{i}",
        "text": chunk_text,
        "source": "class10phy.pdf"
    })

with open("chunk_metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata_json, f, indent=2, ensure_ascii=False)

# --- Save to FAISS vector store ---
vector_store = FAISS.from_documents(split_docs, embedding=embedding_model)
vector_store.save_local(folder_path='/content', index_name='class_10_phy')

# --- Load vector store and metadata ---
vector_store = FAISS.load_local(
    folder_path='/content',
    index_name='class_10_phy',
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

with open("chunk_metadata.json", "r", encoding="utf-8") as f:
    chunk_metadata = json.load(f)
metadata_lookup = {entry["text"]: entry for entry in chunk_metadata}

# --- Query and run through LLM ---
query = "what are the 2 laws of reflection?"
docs: list[Document] = retriever.invoke(query)
relevant_chunks = [doc.page_content for doc in docs]

prompt = """You are given the query from the user, along with the content extracted from the vector embeddings related to the query.
Using this, give the answer to the query.
These are questions related to a physics chapter of class 10.
Give the response in a complete sentence based on the sentence from the context provided.
User Query: {query}
Relevant Content: {relevant_docs}
If the given content doesn't include the answer to the query, respond with 'I don't know'."""

new_prompt = ChatPromptTemplate.from_template(prompt)
chain = new_prompt | llm
response = chain.invoke({"query": query, "relevant_docs": relevant_chunks})

# --- Print output ---
print("✅ Answer:", response.content)
print("\n🔍 Chunks used:\n" + "-" * 60)

for chunk_text in relevant_chunks:
    match = metadata_lookup.get(chunk_text)
    if match:
        print(f"🆔 Chunk ID: {match['chunk_id']}")
        print(f"📄 Page: {match['page']}")
        print(f"📄 Text: {match['text']}\n{'-' * 60}")


✅ Answer: The two laws of reflection are: (i) the angle of incidence is equal to the angle of reflection, and (ii) the incident ray, the normal to the mirror at the point of incidence and the reflected ray, all lie in the same plane.

🔍 Chunks used:
------------------------------------------------------------
🆔 Chunk ID: page_2_chunk_7
📄 Page: 2
📄 Text: Light – Reflection and Refraction
135
Let us recall these laws –
(i)
The angle of incidence is equal to the angle of reflection, and
(ii)
The incident ray, the normal to the mirror at the point of incidence
and the reflected ray, all lie in the same plane.
These laws of reflection are applicable to all types of reflecting surfaces
including spherical surfaces. You are familiar with the formation of image
by a plane mirror. What are the properties of the image? Image formed
------------------------------------------------------------
🆔 Chunk ID: page_25_chunk_142
📄 Page: 25
📄 Text: What you have learnt
n
Light seems to travel in straight

In [3]:
%pip install spacy
%python -m spacy download en_core_web_sm


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [ ]:
import uuid
import json
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document

# 1️⃣ Load PDF pages
docs = PyMuPDFLoader("class10phy.pdf").load()

# 2️⃣ Split into sentence-aware chunks via SpacyTextSplitter
text_splitter = SpacyTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)

# 3️⃣ Add UUIDs + metadata
sentence_metadata = []
uuid_lookup = {}
processed_docs = []

for doc in split_docs:
    uid = str(uuid.uuid4())
    page = doc.metadata.get("page", 0) + 1
    metadata = {**doc.metadata, "uuid": uid, "page": page, "source": "class10phy.pdf"}

    processed_docs.append(Document(page_content=doc.page_content, metadata=metadata))

    entry = {
        "uuid": uid,
        "page": page,
        "text": doc.page_content,
        "source": "class10phy.pdf"
    }

    sentence_metadata.append(entry)
    uuid_lookup[uid] = entry

# 🔒 Save metadata
with open("sentence_metadata.json", "w", encoding="utf-8") as f:
    json.dump(sentence_metadata, f, indent=2, ensure_ascii=False)

with open("uuid_lookup.json", "w", encoding="utf-8") as f:
    json.dump(uuid_lookup, f, indent=2, ensure_ascii=False)

# 4️⃣ Create and reload FAISS vector store
vector_store = FAISS.from_documents(processed_docs, embedding=embedding_model)
vector_store.save_local(folder_path="vector_index", index_name="class_10_phy")

vector_store = FAISS.load_local(
    folder_path="vector_index",
    index_name="class_10_phy",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 5️⃣ Run query
query = "what are the two laws of reflection?"
retrieved = retriever.invoke(query)
relevant = [doc.page_content for doc in retrieved]

# 6️⃣ Prompt LLM for answer
prompt = """You are given the query from the user, along with the content extracted from the vector embeddings related to the query.
Using this, give the answer to the query.
These are questions related to a physics chapter of class 10.
Give the response in a complete sentence based on the sentence from the context provided.
User Query: {query}
Relevant Content: {relevant_docs}
If the given content doesn't include the answer to the query, respond with 'I don't know'."""

# 7️⃣ Load lookup and print results
print("✅ Answer:", response.content)
print("\n🔍 Supporting Sentences:\n" + "-" * 60)

with open("uuid_lookup.json", "r", encoding="utf-8") as f:
    uuid_lookup = json.load(f)

for doc in retrieved:
    uid = doc.metadata.get("uuid")
    entry = uuid_lookup.get(uid)
    if entry:
        print(f"🆔 {uid} | 📄 Page: {entry['page']}")
        print(entry["text"] + "\n" + "-" * 60)


# 2nd LLM trial

In [ ]:
import uuid
import json
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document

# 1️⃣ Load PDF pages
docs = PyMuPDFLoader("class10phy.pdf").load()
# 2️⃣ Split into sentence-aware chunks via SpacyTextSplitter
text_splitter = SpacyTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)

# 3️⃣ Add UUIDs + metadata
sentence_metadata = []
uuid_lookup = {}
processed_docs = []

for doc in split_docs:
    uid = str(uuid.uuid4())
    page = doc.metadata.get("page", 0) + 1
    metadata = {**doc.metadata, "uuid": uid, "page": page, "source": "class10phy.pdf"}

    processed_docs.append(Document(page_content=doc.page_content, metadata=metadata))

    entry = {
        "uuid": uid,
        "page": page,
        "text": doc.page_content,
        "source": "class10phy.pdf"
    }

    sentence_metadata.append(entry)
    uuid_lookup[uid] = entry

# 🔒 Save metadata
with open("sentence_metadata.json", "w", encoding="utf-8") as f:
    json.dump(sentence_metadata, f, indent=2, ensure_ascii=False)

with open("uuid_lookup.json", "w", encoding="utf-8") as f:
    json.dump(uuid_lookup, f, indent=2, ensure_ascii=False)

# 4️⃣ Create and reload FAISS vector store
vector_store = FAISS.from_documents(processed_docs, embedding=embedding_model)
vector_store.save_local(folder_path="vector_index", index_name="class_10_phy")

vector_store = FAISS.load_local(
    folder_path="vector_index",
    index_name="class_10_phy",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 5️⃣ Run query
query = "what are the two laws of reflection?"
retrieved = retriever.invoke(query)
relevant = [doc.page_content for doc in retrieved]

# 6️⃣ Prompt LLM for answer
prompt = """You are given the query from the user, along with the content extracted from the vector embeddings related to the query.
Using this, give the answer to the query.
These are questions related to a physics chapter of class 10.
Give the response in a complete sentence based on the sentence from the context provided.
User Query: {query}
Relevant Content: {relevant_docs}
If the given content doesn't include the answer to the query, respond with 'I don't know'."""
qa_chain = ChatPromptTemplate.from_template(prompt) | llm
response = qa_chain.invoke({"query": query, "relevant_docs": relevant})

# 7️⃣ Load lookup and print results
print("✅ Answer:", response.content)
print("\n🔍 Supporting Chunks:\n" + "-" * 60)

with open("uuid_lookup.json", "r", encoding="utf-8") as f:
    uuid_lookup = json.load(f)

for doc in retrieved:
    uid = doc.metadata.get("uuid")
    entry = uuid_lookup.get(uid)
    if entry:
        print(f"🆔 {uid} | 📄 Page: {entry['page']}")
        print(entry["text"] + "\n" + "-" * 60)


In [ ]:
import uuid
import spacy
import json
import re
import ast
from langchain.prompts import ChatPromptTemplate

# 🧠 Load spaCy for sentence splitting
nlp = spacy.load("en_core_web_sm")

# 1️⃣ Break retrieved chunks into individual sentences with sentence-level UUIDs
sentence_candidates = []
sentence_lookup = {}

for doc in retrieved:
    chunk_uuid = doc.metadata["uuid"]
    page = doc.metadata["page"]
    source = doc.metadata["source"]
    chunk_text = doc.page_content

    for sent in nlp(chunk_text).sents:
        sentence_text = sent.text.strip()
        if not sentence_text:
            continue

        sentence_uuid = str(uuid.uuid4())

        sentence_candidates.append({
            "uuid": sentence_uuid,
            "sentence": sentence_text
        })

        sentence_lookup[sentence_uuid] = {
            "sentence": sentence_text,
            "page": page,
            "chunk_uuid": chunk_uuid,
            "source": source
        }

# 🔒 Save sentence-level metadata for lookup
with open("sentence_lookup.json", "w", encoding="utf-8") as f:
    json.dump(sentence_lookup, f, indent=2, ensure_ascii=False)

# 2️⃣ Format prompt for second LLM pass
formatted_sentences = "\n".join(
    f"- [uuid: {s['uuid']}] {s['sentence']}" for s in sentence_candidates
)

# 🖨️ Debug: Print everything we're passing to the second LLM

print("\n🔹 User Query:")
print(query)

print("\n🔹 Answer from First LLM:")
print(response.content)

print("\n🔹 Candidate Sentences:")
for s in sentence_candidates:
    print(f"[uuid: {s['uuid']}] {s['sentence']}")


justification_prompt = """
You are helping a student understand a textbook answer.

Below is a user question and the answer given by an AI.

You are also given a list of textbook sentences, each with a unique UUID.

Your task is to return only the UUIDs of the sentences that support the answer.

⚠️ Respond ONLY with a JSON list like: ["uuid1", "uuid2"]. No explanation. No markdown.

---

Question: {query}
Answer: {answer}

Candidate Sentences:
{sentences}
"""

justification_chain = ChatPromptTemplate.from_template(justification_prompt) | llm

justification_response = justification_chain.invoke({
    "query": query,
    "answer": response.content,
    "sentences": formatted_sentences
})

# 3️⃣ Parse returned UUIDs safely
raw_output = justification_response.content.strip()

# Clean code block if needed
if raw_output.startswith("```"):
    raw_output = re.sub(r"^```(?:json)?\n?", "", raw_output)
    raw_output = re.sub(r"\n?```$", "", raw_output)

try:
    returned_uuids = ast.literal_eval(raw_output)
    if not isinstance(returned_uuids, list):
        raise ValueError("Output is not a list")
except Exception as e:
    print("⚠️ Could not parse LLM output:", raw_output)
    returned_uuids = []

# 4️⃣ Lookup and print only the supporting sentences
print("📚 Sentences to Highlight:\n" + "=" * 50)
for uid in returned_uuids:
    entry = sentence_lookup.get(uid)
    if entry:
        print(f"📄 Page: {entry['page']}")
        print(f"📝 {entry['sentence']}")
        print("-" * 50)


In [ ]:
from syntok import segmenter
import json
import re
import ast
from langchain.prompts import ChatPromptTemplate

# 1️⃣ Sentence splitting using syntok
sentence_candidates = []
sentence_lookup = {}
sentence_index = 1

for doc in retrieved:
    chunk_uuid = doc.metadata["uuid"]
    page = doc.metadata["page"]
    source = doc.metadata["source"]
    chunk_text = doc.page_content

    # Process chunk with syntok
    for paragraph in segmenter.process(chunk_text):
        for sentence in paragraph:
            sentence_text = " ".join(token.value for token in sentence).strip()
            if not sentence_text:
                continue

            sentence_candidates.append({
                "index": sentence_index,
                "sentence": sentence_text,
                "page": page,
                "chunk_uuid": chunk_uuid,
                "source": source
            })

            sentence_lookup[sentence_index] = {
                "sentence": sentence_text,
                "page": page,
                "chunk_uuid": chunk_uuid,
                "source": source
            }

            sentence_index += 1

# 🔒 Save for later use
with open("sentence_lookup.json", "w", encoding="utf-8") as f:
    json.dump(sentence_lookup, f, indent=2, ensure_ascii=False)

# 🖨️ Print for debugging
print("\n🔹 User Query:")
print(query)
print("\n🔹 Answer from First LLM:")
print(response.content)
print("\n🔹 Candidate Sentences:")
for s in sentence_candidates:
    print(f"[#{s['index']}] {s['sentence']}")

# 2️⃣ Prepare LLM prompt
formatted_sentences = "\n".join(
    f"- [#{s['index']}] {s['sentence']}" for s in sentence_candidates
)

justification_prompt = """
You are helping a student understand a textbook answer.

Below is a user question and the AI's answer.

You are also given a list of textbook sentences, each numbered like [#1], [#2], etc.

Return only the sentence numbers that support the answer.

⚠️ Respond ONLY with a JSON list of numbers. Example: [1, 4, 7]

---

Question: {query}
Answer: {answer}

Candidate Sentences:
{sentences}
"""

# 3️⃣ Invoke LLM
justification_chain = ChatPromptTemplate.from_template(justification_prompt) | llm
justification_response = justification_chain.invoke({
    "query": query,
    "answer": response.content,
    "sentences": formatted_sentences
})

# 4️⃣ Parse the output
raw_output = justification_response.content.strip()
if raw_output.startswith("```"):
    raw_output = re.sub(r"^```(?:json)?\n?", "", raw_output)
    raw_output = re.sub(r"\n?```$", "", raw_output)

try:
    sentence_numbers = ast.literal_eval(raw_output)
    if not isinstance(sentence_numbers, list):
        raise ValueError("Output is not a list")
except Exception as e:
    print("⚠️ Could not parse LLM output:", raw_output)
    sentence_numbers = []

# 5️⃣ Show final highlighted sentences
print("\n📚 Sentences to Highlight:\n" + "=" * 50)
for idx in sentence_numbers:
    entry = sentence_lookup.get(idx)
    if entry:
        print(f"📄 Page: {entry['page']}")
        print(f"📝 {entry['sentence']}")
        print("-" * 50)


In [ ]:

%pip install syntok


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 1.2 MB/s eta 0:00:11
     -- ------------------------------------- 0.8/12.8 MB 1.2 MB/s eta 0:00:11
     -- ------------------------------------- 0.8/12.8 MB 1.2 MB/s eta 0:00:11
     --- ------------------------------------ 1.0/12.8 MB 1.2 MB/s eta 0:00:11
     ---- ----------------------------------- 1.3/12.8 MB 1.2 MB/s eta 0:00:10
     ---- ----------------------------------- 1.6/12.8 MB 1.2 MB/s eta 0:00:10
     ----- ---------------------------------- 1.8/12.8 MB 1.2 MB/s eta 0:00:10
     ------ --------------------------------- 2.1/12.8 MB 1.2 MB/s eta 0:00:10
     ------- -------------------------------- 2.4/12.8 MB 1.2 MB/s eta 0:00:09
     -------- ------------------------------- 2.6/12.8 MB 1.2 MB/s eta 0:00:09
     --------- ------------------------------ 2.9/12.8 MB 1.2 MB/s


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# trial 3 (Spacy)

In [ ]:
import uuid
import json
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document

# 1️⃣ Load PDF pages
docs = PyMuPDFLoader("class10phy.pdf").load()

# 2️⃣ Split into overlapping chunks (not sentence-level)
text_splitter = SpacyTextSplitter(chunk_size=1500, chunk_overlap=375)
split_docs = text_splitter.split_documents(docs)

# 3️⃣ Add UUIDs + metadata to chunks
chunk_metadata = []
uuid_lookup = {}
processed_docs = []

for doc in split_docs:
    uid = str(uuid.uuid4())
    page = doc.metadata.get("page", 0) + 1
    metadata = {**doc.metadata, "uuid": uid, "page": page, "source": "class10phy.pdf"}

    processed_docs.append(Document(page_content=doc.page_content, metadata=metadata))

    entry = {
        "uuid": uid,
        "page": page,
        "text": doc.page_content,
        "source": "class10phy.pdf"
    }

    chunk_metadata.append(entry)
    uuid_lookup[uid] = entry

# 🔒 Save chunk metadata
with open("sentence_metadata.json", "w", encoding="utf-8") as f:
    json.dump(chunk_metadata, f, indent=2, ensure_ascii=False)

with open("uuid_lookup.json", "w", encoding="utf-8") as f:
    json.dump(uuid_lookup, f, indent=2, ensure_ascii=False)

# 4️⃣ Create and reload FAISS vector store
vector_store = FAISS.from_documents(processed_docs, embedding=embedding_model)
vector_store.save_local(folder_path="vector_index", index_name="class_10_phy")

vector_store = FAISS.load_local(
    folder_path="vector_index",
    index_name="class_10_phy",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 5️⃣ Run query
query = "what are the different sign convertions for reflection in spherical mirrors?"
retrieved = retriever.invoke(query)
relevant = [doc.page_content for doc in retrieved]

prompt = """
You are helping a Class 10 Physics student answer a textbook-based question.

You are given a set of extracted text chunks from the textbook that may contain the answer.

Your task is to write a complete and accurate answer to the question using **only the information provided** in these chunks.

If the answer depends on specific definitions, formulas, rules, steps, or listed conventions — make sure to **include all of them** clearly and accurately.

You may also include any additional relevant points found in the provided content, **as long as you do not skip or miss any part that is essential to correctly answering the question**.

Do not invent or assume anything beyond what is given. If the content does not contain enough information to answer the question, respond only with: "I don't know".

---

📘 Question:
{query}

📚 Extracted Text:
{relevant_docs}
"""
qa_chain = ChatPromptTemplate.from_template(prompt) | llm
response = qa_chain.invoke({"query": query, "relevant_docs": relevant})

# 7️⃣ Print output
print("✅ Answer:", response.content)
print("\n🔍 Supporting Chunks:\n" + "-" * 60)

for doc in retrieved:
    uid = doc.metadata.get("uuid")
    entry = uuid_lookup.get(uid)
    if entry:
        print(f"🆔 {uid} | 📄 Page: {entry['page']}")
        print(entry["text"] + "\n" + "-" * 60)


c:\Users\Ashish\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\pipeline\lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


✅ Answer: The sign conventions for reflection by spherical mirrors, according to the New Cartesian Sign Convention, are as follows:

1.  The object is always placed to the left of the mirror, implying that light from the object falls on the mirror from the left-hand side.
2.  All distances are measured from the pole (P) of the mirror. The pole is taken as the origin.
3.  All distances parallel to the principal axis are measured from the pole of the mirror. The principal axis of the mirror is taken as the x-axis of the coordinate system.
4.  Distances measured to the right of the origin (along + x-axis) are taken as positive, while those measured to the left of the origin (along – x-axis) are taken as negative.
5.  Distances measured perpendicular to and above the principal axis (along + y-axis) are taken as positive.
6.  Distances measured perpendicular to and below the principal axis (along – y-axis) are taken as negative.

Additionally:

*   The height of the object is taken to be po

In [ ]:
# import spacy
# import json
# from langchain.prompts import ChatPromptTemplate
# from langchain.output_parsers import PydanticOutputParser
# from pydantic import BaseModel
# from typing import List

# # 🧠 Load spaCy for sentence splitting
# nlp = spacy.load("en_core_web_sm")

# # 1️⃣ Break retrieved chunks into individual sentences with metadata
# sentence_candidates = []
# sentence_lookup = {}
# sid_counter = 1

# for doc in retrieved:
#     chunk_uuid = doc.metadata["uuid"]
#     page = doc.metadata["page"]
#     source = doc.metadata["source"]
#     chunk_text = doc.page_content

#     for sent in nlp(chunk_text).sents:
#         sentence_text = sent.text.strip()
#         if not sentence_text:
#             continue

#         sid = f"s{sid_counter}"
#         sid_counter += 1

#         sentence_entry = {
#             "sid": sid,
#             "sentence": sentence_text,
#             "page": page,
#             "chunk_uuid": chunk_uuid,
#             "source": source
#         }

#         sentence_candidates.append({
#             "sid": sid,
#             "sentence": sentence_text
#         })

#         sentence_lookup[sid] = sentence_entry

# # 🔒 Save sentence-level metadata
# with open("sentence_lookup.json", "w", encoding="utf-8") as f:
#     json.dump(sentence_lookup, f, indent=2, ensure_ascii=False)

# # 2️⃣ Format sentences for the LLM
# formatted_sentences = "\n".join(
#     f"- [sid: {s['sid']}] {s['sentence']}" for s in sentence_candidates
# )

# # 3️⃣ Define Pydantic schema
# class SupportIDs(BaseModel):
#     ids: List[str]

# # 4️⃣ Create output parser
# parser = PydanticOutputParser(pydantic_object=SupportIDs)
# format_instructions = parser.get_format_instructions()

# # 5️⃣ Define prompt with format instructions
# prompt = ChatPromptTemplate.from_template("""
# You are helping a student understand a textbook answer.

# You are given:
# - A user question
# - The AI-generated answer to that question
# - A list of textbook sentences, each with a short ID

# Your task is to return only the sentence IDs that support the answer.

# {format_instructions}

# ---

# Question: {query}
# Answer: {answer}

# Candidate Sentences:
# {sentences}
# """)

# # 6️⃣ Run the structured chain
# structured_chain = prompt | llm | parser

# result = structured_chain.invoke({
#     "query": query,
#     "answer": response.content,
#     "sentences": formatted_sentences,
#     "format_instructions": format_instructions
# })

# returned_sids = result.ids

# # ✅ Final Output
# print("\n✅ Supporting Sentence IDs:\n" + "=" * 50)
# print(returned_sids)

# print("\n📝 Sentences to Highlight:\n" + "=" * 50)
# for sid in returned_sids:
#     entry = sentence_lookup.get(sid)
#     if entry:
#         print(f"{sid} (Page {entry['page']}): {entry['sentence']}")


In [ ]:
import spacy
import json
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from typing import List
from math import ceil

# 🧠 Load spaCy
nlp = spacy.load("en_core_web_sm")

# 1️⃣ Sentence segmentation
sentence_candidates = []
sentence_lookup = {}
sid_counter = 1

for doc in retrieved:
    chunk_uuid = doc.metadata["uuid"]
    page = doc.metadata["page"]
    source = doc.metadata["source"]
    chunk_text = doc.page_content

    for sent in nlp(chunk_text).sents:
        sentence_text = sent.text.strip()
        if not sentence_text:
            continue

        sid = f"s{sid_counter}"
        sid_counter += 1

        entry = {
            "sid": sid,
            "sentence": sentence_text,
            "page": page,
            "chunk_uuid": chunk_uuid,
            "source": source
        }

        sentence_candidates.append({"sid": sid, "sentence": sentence_text})
        sentence_lookup[sid] = entry

# 🔒 Save sentence metadata
with open("sentence_lookup.json", "w", encoding="utf-8") as f:
    json.dump(sentence_lookup, f, indent=2, ensure_ascii=False)

# 2️⃣ Schema
class SupportIDs(BaseModel):
    ids: List[str]

parser = PydanticOutputParser(pydantic_object=SupportIDs)
format_instructions = parser.get_format_instructions()

# 3️⃣ Prompt template
prompt = ChatPromptTemplate.from_template("""
You are helping a student understand a textbook answer.

You are given:
- A user question
- The AI-generated answer to that question
- A list of textbook sentences, each with a short ID

Your task is to return only the sentence IDs that support the answer.

{format_instructions}

---

Question: {query}
Answer: {answer}

Candidate Sentences:
{sentences}
""")

# 4️⃣ Process in batches (to avoid context overload)
batch_size = 20
num_batches = ceil(len(sentence_candidates) / batch_size)
all_ids = []

for i in range(num_batches):
    batch = sentence_candidates[i * batch_size:(i + 1) * batch_size]
    formatted_sentences = "\n".join(
        f"- [sid: {s['sid']}] {s['sentence']}" for s in batch
    )

    structured_chain = prompt | llm | parser

    try:
        result = structured_chain.invoke({
            "query": query,
            "answer": response.content,
            "sentences": formatted_sentences,
            "format_instructions": format_instructions
        })

        # Normalize and collect IDs
        cleaned = [sid.replace("sid:", "").strip() for sid in result.ids]
        all_ids.extend(cleaned)

    except Exception as e:
        print(f"⚠️ Batch {i+1} failed: {e}")
        continue

# ✅ Clean final IDs
returned_sids = sorted(set(all_ids), key=lambda x: int(x[1:]))

# ✅ Output
print("\n✅ Supporting Sentence IDs:\n" + "=" * 50)
print(returned_sids)

print("\n📝 Sentences to Highlight:\n" + "=" * 50)
for sid in returned_sids:
    entry = sentence_lookup.get(sid)
    if entry:
        print(f"{sid} (Page {entry['page']}): {entry['sentence']}")



✅ Supporting Sentence IDs:
['s1', 's19', 's20', 's23', 's24', 's25', 's26', 's27', 's28', 's36', 's37', 's38', 's39', 's40', 's41']

📝 Sentences to Highlight:
s1 (Page 10): Light – Reflection and Refraction
143
The New Cartesian Sign Convention described above is illustrated in
Fig.9.9 for your reference.
s19 (Page 9): 9.2.3 Sign Convention for Reflection by Spherical Mirrors


While dealing with the reflection of light by spherical mirrors, we shall
follow a set of sign conventions called the New Cartesian Sign
Convention.
s20 (Page 9): In this convention, the pole (P) of the mirror is taken as the
origin (Fig. 9.9).
s23 (Page 9): The object is always placed to the left of the mirror.
s24 (Page 9): This implies
that the light from the object falls on the mirror from the left-hand
side.
s25 (Page 9): (ii)
All distances parallel to the principal axis are measured from the
pole of the mirror.
s26 (Page 9): (iii)
All the distances measured to the right of the origin (along
+ x-axis) are 

# Trial 3 (Syntok)

In [ ]:
%pip install syntok

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import uuid
import json
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document

# 1️⃣ Load PDF pages
# docs = PyMuPDFLoader("class10phy.pdf").load()
docs = PyMuPDFLoader("geography.pdf").load()

# 2️⃣ Split into overlapping chunks (not sentence-level)
text_splitter = SpacyTextSplitter(chunk_size=1500, chunk_overlap=375)
split_docs = text_splitter.split_documents(docs)

# 3️⃣ Add UUIDs + metadata to chunks
chunk_metadata = []
uuid_lookup = {}
processed_docs = []

for doc in split_docs:
    uid = str(uuid.uuid4())
    page = doc.metadata.get("page", 0) + 1
    # metadata = {**doc.metadata, "uuid": uid, "page": page, "source": "class10phy.pdf"}
    metadata = {**doc.metadata, "uuid": uid, "page": page, "source": "geography.pdf"}

    processed_docs.append(Document(page_content=doc.page_content, metadata=metadata))

    entry = {
        "uuid": uid,
        "page": page,
        "text": doc.page_content,
        # "source": "class10phy.pdf"
        "source": "geography.pdf"
    }

    chunk_metadata.append(entry)
    uuid_lookup[uid] = entry

# 🔒 Save chunk metadata
with open("sentence_metadata.json", "w", encoding="utf-8") as f:
    json.dump(chunk_metadata, f, indent=2, ensure_ascii=False)

with open("uuid_lookup.json", "w", encoding="utf-8") as f:
    json.dump(uuid_lookup, f, indent=2, ensure_ascii=False)

# 4️⃣ Create and reload FAISS vector store
vector_store = FAISS.from_documents(processed_docs, embedding=embedding_model)
# vector_store.save_local(folder_path="vector_index", index_name="class_10_phy")
vector_store.save_local(folder_path="vector_index", index_name="geography_vector")

vector_store = FAISS.load_local(
    folder_path="vector_index",
    # index_name="class_10_phy",
    index_name="geography_vector",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 5️⃣ Run query
# query = "Tell me about where the ganga river originates"
query = "What are the main crops in brazil?"
retrieved = retriever.invoke(query)
relevant = [doc.page_content for doc in retrieved]

prompt = """
You are helping a Class 10 Geography student answer a textbook-based question.

You are given a set of extracted text chunks from the textbook that may contain the answer.

Your task is to write a complete and accurate answer to the question using **only the information provided** in these chunks.

If the answer depends on specific definitions, formulas, rules, steps, or listed conventions — make sure to **include all of them** clearly and accurately.

You may also include any additional relevant points found in the provided content, **as long as you do not skip or miss any part that is essential to correctly answering the question**.

Do not invent or assume anything beyond what is given. If the content does not contain enough information to answer the question, respond only with: "I don't know".

---

📘 Question:
{query}

📚 Extracted Text:
{relevant_docs}
"""
qa_chain = ChatPromptTemplate.from_template(prompt) | llm
response = qa_chain.invoke({"query": query, "relevant_docs": relevant})

# 7️⃣ Print output
print("✅ Answer:", response.content)
print("\n🔍 Supporting Chunks:\n" + "-" * 60)

for doc in retrieved:
    uid = doc.metadata.get("uuid")
    entry = uuid_lookup.get(uid)
    if entry:
        print(f"🆔 {uid} | 📄 Page: {entry['page']}")
        print(entry["text"] + "\n" + "-" * 60)


c:\Users\Ashish\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\pipeline\lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


✅ Answer: In Brazil, agriculture is the main occupation of people living in the highlands and coastal areas. Favorable climate and topography allow for growing a variety of crops. The main cereal crops are rice and maize, with maize production largely concentrated in the central part. Commercial crops like coffee, cocoa, rubber, soyabean, and sugarcane are cultivated on a large scale. Brazil is the largest exporter of coffee and soyabean in the world. The major states growing coffee are Minas Gerais and Sao Paulo. Besides these crops, fruits like bananas, pineapples, oranges, and other citrus fruits are also produced.

🔍 Supporting Chunks:
------------------------------------------------------------
🆔 fb0a8fb9-b52f-40aa-ae61-589cf94426f9 | 📄 Page: 65
54
Geographical explanation
Agriculture : In Brazil, agriculture is the main 
occupation of the people living in the highlands 
and coastal areas.

Favourable climate and 
topography make it possible for growing a 
variety of crops.

Rice 

In [ ]:
import syntok.segmenter as segmenter
import json
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from typing import List
from math import ceil

# 1️⃣ Sentence segmentation using syntok
sentence_candidates = []
sentence_lookup = {}
sid_counter = 1

for doc in retrieved:
    chunk_uuid = doc.metadata["uuid"]
    page = doc.metadata["page"]
    source = doc.metadata["source"]
    chunk_text = doc.page_content

    # Use syntok to break into paragraphs and sentences
    paragraphs = segmenter.analyze(chunk_text)
    for paragraph in paragraphs:
        for sentence in paragraph:
            sentence_text = " ".join(token.value for token in sentence).strip()
            if not sentence_text:
                continue

            sid = f"s{sid_counter}"
            sid_counter += 1

            entry = {
                "sid": sid,
                "sentence": sentence_text,
                "page": page,
                "chunk_uuid": chunk_uuid,
                "source": source
            }

            sentence_candidates.append({"sid": sid, "sentence": sentence_text})
            sentence_lookup[sid] = entry

# 🔒 Save sentence metadata
with open("sentence_lookup.json", "w", encoding="utf-8") as f:
    json.dump(sentence_lookup, f, indent=2, ensure_ascii=False)

# 2️⃣ Schema for LLM response
class SupportIDs(BaseModel):
    ids: List[str]

parser = PydanticOutputParser(pydantic_object=SupportIDs)
format_instructions = parser.get_format_instructions()

# 3️⃣ Prompt for justification
prompt = ChatPromptTemplate.from_template("""
You are helping a student understand which textbook sentences support a given answer.

You are given:
- A user question
- An answer to that question
- A list of textbook sentences, each with a short ID

Your task is to identify the **minimum set** of sentence IDs that directly support the answer.

Only include:
- Sentences that clearly explain the final answer
- Definitions, laws, rules, or steps that are explicitly used in the answer

Do NOT include:
- General background statements
- Sentences that are only loosely related
- Sentences that repeat the same content with different wording

If multiple sentences say the same thing, prefer the **clearest and most complete** one.

{format_instructions}

---

Question: {query}
Answer: {answer}

Candidate Sentences:
{sentences}
""")


# 4️⃣ Process in batches
batch_size = 20
num_batches = ceil(len(sentence_candidates) / batch_size)
all_ids = []

for i in range(num_batches):
    batch = sentence_candidates[i * batch_size:(i + 1) * batch_size]
    formatted_sentences = "\n".join(
        f"- [sid: {s['sid']}] {s['sentence']}" for s in batch
    )

    structured_chain = prompt | llm | parser

    try:
        result = structured_chain.invoke({
            "query": query,
            "answer": response.content,
            "sentences": formatted_sentences,
            "format_instructions": format_instructions
        })

        # Normalize and collect IDs
        cleaned = [sid.replace("sid:", "").strip() for sid in result.ids]
        all_ids.extend(cleaned)

    except Exception as e:
        print(f"⚠️ Batch {i+1} failed: {e}")
        continue

# ✅ Final cleanup
returned_sids = sorted(set(all_ids), key=lambda x: int(x[1:]))

# ✅ Output
print("\n✅ Supporting Sentence IDs:\n" + "=" * 50)
print(returned_sids)

print("\n📝 Sentences to Highlight:\n" + "=" * 50)
for sid in returned_sids:
    entry = sentence_lookup.get(sid)
    if entry:
        print(f"{sid} (Page {entry['page']}): {entry['sentence']}")

with open("highlight_ids.json", "w", encoding="utf-8") as f:
    json.dump(returned_sids, f, indent=2)




✅ Supporting Sentence IDs:
['s1', 's2', 's3', 's5', 's8', 's51', 's52']

📝 Sentences to Highlight:
s1 (Page 65): 54 Geographical explanation Agriculture : In Brazil , agriculture is the main occupation of the people living in the highlands and coastal areas .
s2 (Page 65): Favourable climate and topography make it possible for growing a variety of crops .
s3 (Page 65): Rice and maize are the main cereal crops .
s5 (Page 65): Commercial crops like coffee , cocoa , rubber , soyabean and sugarcane are cultivated on a large scale .
s8 (Page 65): Besides these crops , production of fruits like bananas , Figure 8.3 pineapples , oranges and other citrus fruits is also done .
s51 (Page 67): India produces rice , wheat , maize , sorghum and millets as major food crops ; plantations of tea , coffee , rubber and cash crops like sugarcane , cotton , jute , etc are also produced .
s52 (Page 67): India is also a major producer of a variety of fruits and vegetables .


In [ ]:
import fitz  # PyMuPDF
import json
import re
from pathlib import Path

# 📁 Paths
base_dir = Path(".")
highlight_ids_path = base_dir / "highlight_ids.json"
sentence_lookup_path = base_dir / "sentence_lookup.json"
# input_pdf_path = base_dir / "class10phy.pdf"
input_pdf_path = base_dir / "geography.pdf"
output_pdf_path = base_dir / "highlighted_output.pdf"

# 🧹 Normalization helper
def normalize(text):
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip().lower()

# 📄 Load data
with open(highlight_ids_path, "r", encoding="utf-8") as f:
    highlight_ids = json.load(f)

with open(sentence_lookup_path, "r", encoding="utf-8") as f:
    sentence_lookup = json.load(f)

# 📘 Open PDF
doc = fitz.open(input_pdf_path)

# 🟨 Highlight loop
for sid in highlight_ids:
    entry = sentence_lookup[sid]
    sentence = entry["sentence"]
    page = doc[entry["page"] - 1]
    sentence_norm = normalize(sentence)

    # Step 1: get all words with bounding boxes
    words = page.get_text("words")  # list of [x0, y0, x1, y1, "word"]
    words_norm = [normalize(w[4]) for w in words]

    # Step 2: sliding window match
    match_found = False
    for i in range(len(words_norm)):
        for j in range(i + 5, len(words_norm) + 1):  # allow sentences at least 5 words
            phrase = " ".join(words_norm[i:j])
            if phrase == sentence_norm:
                match_found = True
                rects = [fitz.Rect(words[k][0], words[k][1], words[k][2], words[k][3]) for k in range(i, j)]
                for r in rects:
                    page.add_highlight_annot(r)
                print(f"✅ Highlighted: {sid}")
                break
        if match_found:
            break

    if not match_found:
        print(f"❌ Not found: {sentence[:80]}")

# 💾 Save
doc.save(output_pdf_path, garbage=4, deflate=True)
doc.close()
print(f"✅ Highlighted PDF saved to: {output_pdf_path}")


❌ Not found: 54 Geographical explanation Agriculture : In Brazil , agriculture is the main oc
✅ Highlighted: s2
✅ Highlighted: s3
✅ Highlighted: s5
✅ Highlighted: s8
✅ Highlighted: s51
✅ Highlighted: s52
✅ Highlighted PDF saved to: highlighted_output.pdf


In [ ]:
import fitz  # PyMuPDF

def extract_lines_with_line_numbers(pdf_path, page_num=0):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    
    # Extract text blocks from the first page (page_num = 0 for first page)
    page = doc.load_page(page_num)
    text_instances = page.get_text("dict")["blocks"]
    
    lines = []
    line_number = 1  # Start with line 1
    
    for block in text_instances:
        if block['type'] == 0:  # Text block (only type 0 is text)
            for line in block["lines"]:
                line_text = ""
                for span in line["spans"]:
                    line_text += span["text"]  # Concatenate the text from each span
                
                # Append line text with the line number
                lines.append((line_number, line_text.strip()))
                line_number += 1  # Increment line number
    
    return lines

# Path to your PDF file
pdf_path = "class10phy.pdf"

# Extract lines with line numbers from the first page
lines = extract_lines_with_line_numbers(pdf_path, page_num=0)

# Print each line with its line number
for line_number, line_text in lines:
    print(f"Line {line_number}: {line_text}")


Line 1: Science
Line 2: 134
Line 3: Light – Reflection and
Line 4: Refraction
Line 5: 9
Line 6: CHAPTER
Line 7: W
Line 8: e see a variety of objects in the world around us. However, we are
Line 9: unable to see anything in a dark room. On lighting up the room,
Line 10: things become visible. What makes things visible? During the day, the
Line 11: sunlight helps us to see objects. An object reflects light that falls on it.
Line 12: This reflected light, when received by our eyes, enables us to see things.
Line 13: We are able to see through a transparent medium as light is transmitted
Line 14: through it. There are a number of common wonderful phenomena
Line 15: associated with light such as image formation by mirrors, the twinkling
Line 16: of stars, the beautiful colours of a rainbow, bending of light by a medium
Line 17: and so on.  A study of the properties of light helps us to explore them.
Line 18: By observing the common optical phenomena around us, we may
Line 19: conclude that 

## 2 step thinking model and Agentic/ReAct loop

In [ ]:
import uuid
import json
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import Document

# 1️⃣ Load PDF pages
# docs = PyMuPDFLoader("class10phy.pdf").load()
docs = PyMuPDFLoader("geography.pdf").load()

# 2️⃣ Split into overlapping chunks (not sentence-level)
text_splitter = SpacyTextSplitter(chunk_size=1500, chunk_overlap=375)
split_docs = text_splitter.split_documents(docs)

# 3️⃣ Add UUIDs + metadata to chunks
chunk_metadata = []
uuid_lookup = {}
processed_docs = []

for doc in split_docs:
    uid = str(uuid.uuid4())
    page = doc.metadata.get("page", 0) + 1
    # metadata = {**doc.metadata, "uuid": uid, "page": page, "source": "class10phy.pdf"}
    metadata = {**doc.metadata, "uuid": uid, "page": page, "source": "geography.pdf"}

    processed_docs.append(Document(page_content=doc.page_content, metadata=metadata))

    entry = {
        "uuid": uid,
        "page": page,
        "text": doc.page_content,
        # "source": "class10phy.pdf"
        "source": "geography.pdf"
    }

    chunk_metadata.append(entry)
    uuid_lookup[uid] = entry

# 🔒 Save chunk metadata
with open("sentence_metadata.json", "w", encoding="utf-8") as f:
    json.dump(chunk_metadata, f, indent=2, ensure_ascii=False)

with open("uuid_lookup.json", "w", encoding="utf-8") as f:
    json.dump(uuid_lookup, f, indent=2, ensure_ascii=False)

# 4️⃣ Create and reload FAISS vector store
vector_store = FAISS.from_documents(processed_docs, embedding=embedding_model)
# vector_store.save_local(folder_path="vector_index", index_name="class_10_phy")
vector_store.save_local(folder_path="vector_index", index_name="geography_vector")

vector_store = FAISS.load_local(
    folder_path="vector_index",
    # index_name="class_10_phy",
    index_name="geography_vector",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})


c:\Users\Ashish\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\pipeline\lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
# from langchain.prompts import ChatPromptTemplate

# # 5️⃣ Run query
# # query = "Tell me about where the ganga river originates"
# # query = "What are the main crops in brazil?"
# query = "What place in india has the most urbanisation?"
# retrieved = retriever.invoke(query)
# relevant = [doc.page_content for doc in retrieved]

# # Step 1️⃣ Extract facts
# facts_prompt = ChatPromptTemplate.from_template("""
# You are a helpful assistant extracting information from a textbook.

# Task: From the given chunks, extract ONLY the key facts, rules, or definitions 
# that directly answer the question.

# ⚠️ If the question asks for a "most", "highest", or "first", make sure to identify 
# the item with the **highest value** or most relevant occurrence, 
# even if the text mentions a lower value first.

# - Copy text closely from the chunks (do not paraphrase heavily).
# - Skip irrelevant details.
# - If no facts are relevant, return "NONE".

# 📘 Question:
# {query}

# 📚 Extracted Text:
# {relevant_docs}
# """)


# facts_chain = facts_prompt | llm
# facts_response = facts_chain.invoke({"query": query, "relevant_docs": relevant})

# print("🔍 Extracted Facts:\n", facts_response.content)
# print("=" * 80)

# answer_prompt = ChatPromptTemplate.from_template("""
# You are a helpful tutor writing a final answer for a student.

# Use ONLY the extracted facts provided. 
# Do not add any outside knowledge. 
# Do not skip or invent any essential facts.

# Write the answer in a clear, concise, and textbook-like style. 
# Follow the content in the extracted facts closely, 
# but it’s okay to combine sentences for readability. 
# Do not change factual details.

# 📘 Question:
# {query}

# 📝 Extracted Facts:
# {facts}
# """)


# answer_chain = answer_prompt | llm
# answer_response = answer_chain.invoke({"query": query, "facts": facts_response.content})

# print("✅ Final Answer:\n", answer_response.content)
# print("=" * 80)

# # 🔎 Print supporting chunks for transparency
# print("\n🔍 Supporting Chunks:\n" + "-" * 60)
# for doc in retrieved:
#     uid = doc.metadata.get("uuid")
#     entry = uuid_lookup.get(uid)
#     if entry:
#         print(f"🆔 {uid} | 📄 Page: {entry['page']}")
#         print(entry["text"] + "\n" + "-" * 60)


In [ ]:
from langchain.prompts import ChatPromptTemplate

# 5️⃣ Run query
query = "what are the different seasons"
retrieved = retriever.invoke(query)
relevant = [doc.page_content for doc in retrieved]

# STEP 1 — Evidence + Selection (reasoning phase)
reasoning_prompt = ChatPromptTemplate.from_template("""
You are analyzing textbook excerpts to answer a question.

📘 Question:
{query}

📚 Textbook Chunks:
{relevant_docs}

Instructions:
1. Extract **all key facts** relevant to the question.
   - Include numbers, percentages, dates, or terms exactly as written.
   - Include comparative or regional statements if they help.
   - Ignore irrelevant text.
2. Identify the **list of entities** that directly answer the question.
   - Each entity should be a single item (e.g., season, concept, person, measurement).
3. Provide a short **justification** for why these entities are correct, based only on the extracted facts.

Return EXACTLY in this format:

FACTS:
- <fact1>
- <fact2>
...

ENTITIES:
- <entity1>
- <entity2>
...

JUSTIFICATION: <one short sentence based only on the facts>
""")

reasoning_chain = reasoning_prompt | llm
reasoning_response = reasoning_chain.invoke({
    "query": query,
    "relevant_docs": relevant
})

print("🔍 Reasoning Phase:\n", reasoning_response.content)
print("=" * 80)

# STEP 2 — Final Answer (teaching phase)
answer_prompt = ChatPromptTemplate.from_template("""
You are a tutor writing a **clear, textbook-style answer**.

📘 Question:
{query}

🔎 Reasoning Output:
{reasoning}

Instructions:
- Start by clearly listing the entities that answer the question.
- Expand slightly by summarizing the supporting facts in simple sentences.
- Maintain all numbers, names, and phrases exactly as in the reasoning output.
- Keep it concise, readable, and like a textbook explanation for students.
- Do not add outside knowledge or speculation.

Return **only the final answer paragraph**.
""")


answer_chain = answer_prompt | llm
answer_response = answer_chain.invoke({
    "query": query,
    "reasoning": reasoning_response.content
})

print("✅ Final Answer:\n", answer_response.content)
print("=" * 80)

# 🔎 Print supporting chunks for transparency
print("\n🔍 Supporting Chunks:\n" + "-" * 60)
for doc in retrieved:
    uid = doc.metadata.get("uuid")
    entry = uuid_lookup.get(uid)
    if entry:
        print(f"🆔 {uid} | 📄 Page: {entry['page']}")
        print(entry["text"] + "\n" + "-" * 60)


🔍 Reasoning Phase:
 FACTS:
- There are four seasons as per the Indian Meteorological Department.
- The hot weather season
- The season of rainfall (Monsoon)
- The season of Retreating Monsoon
- The cold weather season

ENTITIES:
- hot weather season
- season of rainfall (Monsoon)
- season of Retreating Monsoon
- cold weather season

JUSTIFICATION: The text states that there are four seasons according to the Indian Meteorological Department, and then lists those four seasons.
✅ Final Answer:
 The four seasons, according to the Indian Meteorological Department, are: the hot weather season, the season of rainfall (Monsoon), the season of Retreating Monsoon, and the cold weather season.

🔍 Supporting Chunks:
------------------------------------------------------------
🆔 b5c638cb-0edc-4640-a3ba-b2d8f81bc7d8 | 📄 Page: 40
z
The season of Retreating Monsoon
z
The cold weather season
Observe the pictures given below from 
figure 4.6 to 4.13 and write a brief description 
about them.


Use your 

In [ ]:
import syntok.segmenter as segmenter
import json
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from typing import List
from math import ceil

# 1️⃣ Sentence segmentation using syntok
sentence_candidates = []
sentence_lookup = {}
sid_counter = 1

for doc in retrieved:
    chunk_uuid = doc.metadata["uuid"]
    page = doc.metadata["page"]
    source = doc.metadata["source"]
    chunk_text = doc.page_content

    # Break into paragraphs and sentences
    paragraphs = segmenter.analyze(chunk_text)
    for paragraph in paragraphs:
        for sentence in paragraph:
            sentence_text = " ".join(token.value for token in sentence).strip()
            if not sentence_text:
                continue

            sid = f"s{sid_counter}"
            sid_counter += 1

            entry = {
                "sid": sid,
                "sentence": sentence_text,
                "page": page,
                "chunk_uuid": chunk_uuid,
                "source": source
            }

            sentence_candidates.append({"sid": sid, "sentence": sentence_text})
            sentence_lookup[sid] = entry

# 🔒 Save sentence metadata
with open("sentence_lookup.json", "w", encoding="utf-8") as f:
    json.dump(sentence_lookup, f, indent=2, ensure_ascii=False)

# 2️⃣ Schema for LLM response
class SupportIDs(BaseModel):
    ids: List[str]

parser = PydanticOutputParser(pydantic_object=SupportIDs)
format_instructions = parser.get_format_instructions()

# 3️⃣ Prompt for justification
prompt = ChatPromptTemplate.from_template("""
You are helping a student understand which textbook sentences support a given answer.

You are given:
- A user question
- An answer to that question
- A list of textbook sentences, each with a short ID

Your task is to identify the **minimum set** of sentence IDs that directly support the answer.

Only include:
- Sentences that clearly explain the final answer
- Definitions, laws, rules, or steps that are explicitly used in the answer

Do NOT include:
- General background statements
- Sentences that are only loosely related
- Sentences that repeat the same content with different wording

If multiple sentences say the same thing, prefer the **clearest and most complete** one.

{format_instructions}

---

Question: {query}
Answer: {answer}

Candidate Sentences:
{sentences}
""")

# 4️⃣ Process in batches
batch_size = 20
num_batches = ceil(len(sentence_candidates) / batch_size)
all_ids = []

for i in range(num_batches):
    batch = sentence_candidates[i * batch_size:(i + 1) * batch_size]
    formatted_sentences = "\n".join(
        f"- [sid: {s['sid']}] {s['sentence']}" for s in batch
    )

    structured_chain = prompt | llm | parser

    try:
        result = structured_chain.invoke({
            "query": query,
            "answer": answer_response.content,   # ✅ use final two-step answer
            "sentences": formatted_sentences,
            "format_instructions": format_instructions
        })

        cleaned = [sid.replace("sid:", "").strip() for sid in result.ids]
        all_ids.extend(cleaned)

    except Exception as e:
        print(f"⚠️ Batch {i+1} failed: {e}")
        continue

# ✅ Final cleanup
returned_sids = sorted(set(all_ids), key=lambda x: int(x[1:]))

# ✅ Output
print("\n✅ Supporting Sentence IDs:\n" + "=" * 50)
print(returned_sids)

print("\n📝 Sentences to Highlight:\n" + "=" * 50)
for sid in returned_sids:
    entry = sentence_lookup.get(sid)
    if entry:
        print(f"{sid} (Page {entry['page']}): {entry['sentence']}")

with open("highlight_ids.json", "w", encoding="utf-8") as f:
    json.dump(returned_sids, f, indent=2)



✅ Supporting Sentence IDs:
['s16', 's20', 's21', 's22']

📝 Sentences to Highlight:
s16 (Page 40): This is the Retreating Monsoon season in India .
s20 (Page 40): There are four seasons as per the Indian Meteorological Department .
s21 (Page 40): z The hot weather season z The season of rainfall ( Monsoon )
s22 (Page 40): z The season of Retreating Monsoon z The cold weather season Observe the pictures given below from figure 4.6 to 4.13 and write a brief description about them .


In [ ]:
import fitz  # PyMuPDF
import json
import re
from pathlib import Path

# 📁 Paths
base_dir = Path(".")
highlight_ids_path = base_dir / "highlight_ids.json"
sentence_lookup_path = base_dir / "sentence_lookup.json"
# input_pdf_path = base_dir / "class10phy.pdf"
input_pdf_path = base_dir / "geography.pdf"
output_pdf_path = base_dir / "highlighted_output.pdf"

# 🧹 Normalization helper
def normalize(text):
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip().lower()

# 📄 Load data
with open(highlight_ids_path, "r", encoding="utf-8") as f:
    highlight_ids = json.load(f)

with open(sentence_lookup_path, "r", encoding="utf-8") as f:
    sentence_lookup = json.load(f)

# 📘 Open PDF
doc = fitz.open(input_pdf_path)

# 🟨 Highlight loop
for sid in highlight_ids:
    entry = sentence_lookup[sid]
    sentence = entry["sentence"]
    page = doc[entry["page"] - 1]
    sentence_norm = normalize(sentence)

    # Step 1: get all words with bounding boxes
    words = page.get_text("words")  # list of [x0, y0, x1, y1, "word"]
    words_norm = [normalize(w[4]) for w in words]

    # Step 2: sliding window match
    match_found = False
    for i in range(len(words_norm)):
        for j in range(i + 5, len(words_norm) + 1):  # allow sentences at least 5 words
            phrase = " ".join(words_norm[i:j])
            if phrase == sentence_norm:
                match_found = True
                rects = [fitz.Rect(words[k][0], words[k][1], words[k][2], words[k][3]) for k in range(i, j)]
                for r in rects:
                    page.add_highlight_annot(r)
                print(f"✅ Highlighted: {sid}")
                break
        if match_found:
            break

    if not match_found:
        print(f"❌ Not found: {sentence[:80]}")

# 💾 Save
doc.save(output_pdf_path, garbage=4, deflate=True)
doc.close()
print(f"✅ Highlighted PDF saved to: {output_pdf_path}")


✅ Highlighted: s16
✅ Highlighted: s20
✅ Highlighted: s21
✅ Highlighted: s22
✅ Highlighted PDF saved to: highlighted_output.pdf


## Handling bullets and other special symbols

In [4]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ashish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Ashish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
import uuid
import json
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema import Document

# 1️⃣ Load PDF pages
# docs = PyMuPDFLoader("class10phy.pdf").load()
docs = PyMuPDFLoader("class10phy.pdf").load()

# 2️⃣ Split into overlapping chunks (not sentence-level)
text_splitter = SpacyTextSplitter(chunk_size=1500, chunk_overlap=375)
split_docs = text_splitter.split_documents(docs)

# 3️⃣ Add UUIDs + metadata to chunks
chunk_metadata = []
uuid_lookup = {}
processed_docs = []

for doc in split_docs:
    uid = str(uuid.uuid4())
    page = doc.metadata.get("page", 0) + 1
    # metadata = {**doc.metadata, "uuid": uid, "page": page, "source": "class10phy.pdf"}
    metadata = {**doc.metadata, "uuid": uid, "page": page, "source": "geography.pdf"}

    processed_docs.append(Document(page_content=doc.page_content, metadata=metadata))

    entry = {
        "uuid": uid,
        "page": page,
        "text": doc.page_content,
        "source": "class10phy.pdf"
        # "source": "history.pdf"
        # "source": "geography.pdf"
    }

    chunk_metadata.append(entry)
    uuid_lookup[uid] = entry

# 🔒 Save chunk metadata
with open("sentence_metadata.json", "w", encoding="utf-8") as f:
    json.dump(chunk_metadata, f, indent=2, ensure_ascii=False)

with open("uuid_lookup.json", "w", encoding="utf-8") as f:
    json.dump(uuid_lookup, f, indent=2, ensure_ascii=False)

# 4️⃣ Create and reload FAISS vector store
vector_store = FAISS.from_documents(processed_docs, embedding=embedding_model)
vector_store.save_local(folder_path="vector_index", index_name="class_10_phy")
# vector_store.save_local(folder_path="vector_index", index_name="geography_vector")
# vector_store.save_local(folder_path="vector_index", index_name="history_vector")

vector_store = FAISS.load_local(
    folder_path="vector_index",
    index_name="class_10_phy",
    # index_name="geography_vector",
    # index_name="history_vector",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})


c:\Users\Ashish\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\pipeline\lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [194]:
from langchain.prompts import ChatPromptTemplate

# 5️⃣ Run query
query = "how to calculate refractive index between 2 mediums?"  # replace as needed
retrieved = retriever.invoke(query)
relevant = [doc.page_content for doc in retrieved]

# STEP 1 — Evidence + Selection (reasoning phase)
reasoning_prompt = ChatPromptTemplate.from_template("""
You are analyzing textbook excerpts to answer a question.

📘 Question:
{query}

📚 Textbook Chunks:
{relevant_docs}

Instructions:
1) Extract ALL key facts that directly address the question.
   - Always include the main laws, rules, definitions, named results, and formulas.
   - Also include any interpretation conventions needed to read/apply those items
     (generic rule: if a sign/value/symbol implies a meaning or case, capture that mapping).
   - Include any applicability constraints the source states (domains, ranges, bounds, validity conditions).
   - Include any concise conceptual statement that explains what an item represents or relates
     (e.g., “gives the relationship between …”, “is expressed as the ratio of …”).
   - Keep numbers, symbols, equations, variable names, constants, and technical terms EXACTLY as written.
   - Ignore only text that is clearly unrelated.

2) Create a brief GLOSSARY for items used in those facts:
   - List abbreviations, acronyms, symbols, variables, constants, or units that appear in the facts/equations.
   - For each, give its meaning exactly as stated in the text (precise and minimal).
   - If the text maps a sign/value to a meaning (e.g., “positive means …”), record that mapping once.

3) Identify the list of ENTITIES that directly answer the question.
   - Each entity should be one atomic item (law, rule, definition, condition, or equation).

4) Provide a short JUSTIFICATION based only on the extracted facts.

Return EXACTLY in this format:

FACTS:
- <fact1>
- <fact2>
...

GLOSSARY:
- <token> = <meaning>
- <token> = <meaning>
...

ENTITIES:
- <entity1>
- <entity2>
...

JUSTIFICATION: <one short sentence based only on the facts>
""")

reasoning_chain = reasoning_prompt | llm
reasoning_response = reasoning_chain.invoke({
    "query": query,
    "relevant_docs": relevant
})

print("🔍 Reasoning Phase:\n", reasoning_response.content)
print("=" * 80)

# STEP 2 — Final Answer (self-contained items, adaptive formatting)
answer_prompt = ChatPromptTemplate.from_template("""
You are a tutor writing a clear, textbook-style answer.

📘 Question:
{query}

🔎 Reasoning Output:
{reasoning}

Goal:
Return a polished answer that includes the core item(s) (e.g., formulas/definitions) PLUS the minimal context
needed to understand/apply them. The style should fit the question.

Formatting policy (choose ONE style):
- If the question effectively asks for ONE core item (e.g., a single definition or formula),
  write a compact PARAGRAPH (1-3 sentences). Include the equation inline and any essential
  interpretation conventions or constraints in the same paragraph.
- If there are 2-5 core items, use a clean NUMBERED LIST (1 level only; no sub-bullets unless unavoidable).
- If the question names “laws/rules/steps” or there are >5 items, use a numbered list.

Content rules (apply regardless of style):
- For EACH core item:
  • Include its formal statement/equation verbatim (symbols/notation unchanged).
  • Add one brief generic interpretation (e.g., “gives the relationship between …”, “ratio of …”).
  • Attach any essential interpretation conventions (e.g., sign/value → meaning) and applicability constraints.
  • Inline the minimal meanings of symbols/variables/constants needed to read the item on its own
    (use the provided glossary/facts; keep it brief).
- DO NOT move essentials to “Additional Points.” Essentials must appear with the item.
- Preserve all numbers, symbols, and precise terms from the reasoning facts/glossary.
- Prefer exact phrasing from the excerpts for formal names and equations.
- Avoid verbose sub-lists. One level only. Keep it crisp and readable.

Additional Points (optional):
- Add a section titled exactly: **Additional Points:**
- Include 0–2 short bullets that are helpful but NOT required to interpret/apply the main items
  (e.g., a common pitfall, a tiny tip, or a closely related consequence).
- Do NOT duplicate content already stated in the main section.

Special handling:
- If the question looks like multiple-choice, clearly state the correct option(s) with a brief justification.
- If a length is requested (e.g., “in N words”), match it (±10%) prioritizing facts from the excerpts.
- If explicitly asked to compare/contrast, present a 2-column Markdown table.

Return ONLY the final answer text (main section + the **Additional Points** section, if any).
""")

answer_chain = answer_prompt | llm
answer_response = answer_chain.invoke({
    "query": query,
    "reasoning": reasoning_response.content
})

print("✅ Final Answer:\n", answer_response.content)
print("=" * 80)

# Expose results as true globals for later cells
global final_answer_text, reasoning_text
final_answer_text = answer_response.content
reasoning_text    = reasoning_response.content

# 🔎 Print supporting chunks for transparency
print("\n🔍 Supporting Chunks:\n" + "-" * 60)
for doc in retrieved:
    uid = doc.metadata.get("uuid")
    entry = uuid_lookup.get(uid)
    if entry:
        print(f"🆔 {uid} | 📄 Page: {entry['page']}")
        print(entry["text"] + "\n" + "-" * 60)


🔍 Reasoning Phase:
 FACTS:
- The refractive index of medium 2 with respect to medium 1 is given by the ratio of the speed of light in medium 1 and the speed of light in medium 2.
- n21= Speed of light in medium 1 / Speed of light in medium 2 = v1/v2 (9.5)
- The refractive index of medium 1 with respect to medium 2 is represented as n12.
- n12= Speed of light in medium 2 / Speed of light in medium 1 = v2/v1 (9.6)
- If medium 1 is vacuum or air, then the refractive index of medium 2 is considered with respect to vacuum, called the absolute refractive index of the medium, represented as n2.
- nm = Speed of light in air / Speed of light in the medium = c/v (9.7)

GLOSSARY:
- n21 = refractive index of medium 2 with respect to medium 1
- v1 = speed of light in medium 1
- v2 = speed of light in medium 2
- n12 = refractive index of medium 1 with respect to medium 2
- n2 = absolute refractive index of medium 2
- nm = refractive index of the medium
- c = Speed of light in air
- v = Speed of ligh

In [195]:
# import syntok.segmenter as segmenter
# import json
# from langchain.prompts import ChatPromptTemplate
# from langchain.output_parsers import PydanticOutputParser
# from pydantic import BaseModel
# from typing import List
# from math import ceil

# # 1️⃣ Sentence segmentation using syntok
# sentence_candidates = []
# sentence_lookup = {}
# sid_counter = 1

# for doc in retrieved:
#     chunk_uuid = doc.metadata["uuid"]
#     page = doc.metadata["page"]
#     source = doc.metadata["source"]
#     chunk_text = doc.page_content

#     # Break into paragraphs and sentences
#     paragraphs = segmenter.analyze(chunk_text)
#     for paragraph in paragraphs:
#         for sentence in paragraph:
#             sentence_text = " ".join(token.value for token in sentence).strip()
#             if not sentence_text:
#                 continue

#             sid = f"s{sid_counter}"
#             sid_counter += 1

#             entry = {
#                 "sid": sid,
#                 "sentence": sentence_text,
#                 "page": page,
#                 "chunk_uuid": chunk_uuid,
#                 "source": source
#             }

#             sentence_candidates.append({"sid": sid, "sentence": sentence_text})
#             sentence_lookup[sid] = entry

# # 🔒 Save sentence metadata
# with open("sentence_lookup.json", "w", encoding="utf-8") as f:
#     json.dump(sentence_lookup, f, indent=2, ensure_ascii=False)

# # 2️⃣ Schema for LLM response
# class SupportIDs(BaseModel):
#     ids: List[str]

# parser = PydanticOutputParser(pydantic_object=SupportIDs)
# format_instructions = parser.get_format_instructions()

# # 3️⃣ Prompt for justification
# prompt = ChatPromptTemplate.from_template("""
# You are helping a student understand which textbook sentences support a given answer.

# You are given:
# - A user question
# - An answer to that question
# - A list of textbook sentences, each with a short ID

# Your task is to identify the **minimum set** of sentence IDs that directly support the answer.

# Only include:
# - Sentences that clearly explain the final answer
# - Definitions, laws, rules, or steps that are explicitly used in the answer

# Do NOT include:
# - General background statements
# - Sentences that are only loosely related
# - Sentences that repeat the same content with different wording

# If multiple sentences say the same thing, prefer the **clearest and most complete** one.
# If the same keyword is used is found in more than one place, use it only if it clearly is required to answer the answer accurately,
# this doesn't include the introductory sentences that lead to the main point.

# {format_instructions}

# ---

# Question: {query}
# Answer: {answer}

# Candidate Sentences:
# {sentences}
# """)

# # 4️⃣ Process in batches
# batch_size = 20
# num_batches = ceil(len(sentence_candidates) / batch_size)
# all_ids = []

# for i in range(num_batches):
#     batch = sentence_candidates[i * batch_size:(i + 1) * batch_size]
#     formatted_sentences = "\n".join(
#         f"- [sid: {s['sid']}] {s['sentence']}" for s in batch
#     )

#     structured_chain = prompt | llm | parser

#     try:
#         result = structured_chain.invoke({
#             "query": query,
#             "answer": answer_response.content,   # ✅ use final two-step answer
#             "sentences": formatted_sentences,
#             "format_instructions": format_instructions
#         })

#         cleaned = [sid.replace("sid:", "").strip() for sid in result.ids]
#         all_ids.extend(cleaned)

#     except Exception as e:
#         print(f"⚠️ Batch {i+1} failed: {e}")
#         continue

# # ✅ Final cleanup
# returned_sids = sorted(set(all_ids), key=lambda x: int(x[1:]))

# # ✅ Output
# print("\n✅ Supporting Sentence IDs:\n" + "=" * 50)
# print(returned_sids)

# print("\n📝 Sentences to Highlight:\n" + "=" * 50)
# for sid in returned_sids:
#     entry = sentence_lookup.get(sid)
#     if entry:
#         print(f"{sid} (Page {entry['page']}): {entry['sentence']}")

# with open("highlight_ids.json", "w", encoding="utf-8") as f:
#     json.dump(returned_sids, f, indent=2)


In [196]:
# ===== CELL 3 — v2's repair/merge + v1's minimal LLM selection (with parenthetical stitch) =====
import syntok.segmenter as segmenter
import json, re
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from typing import List
from math import ceil

# --------- repair/merge logic (from your v2, unchanged except for a tiny generic addition) ---------
OP_CHARS = r"<>=±+\-*/^:≈≠≤≥→∝"
OP_TAIL_RE    = re.compile(rf"[{re.escape(OP_CHARS)}]\s*$")           # e.g., "0 <"
OP_HEAD_RE    = re.compile(rf"^\s*[{re.escape(OP_CHARS)}]")           # e.g., "< i"
CLOSE_HEAD_RE = re.compile(r"^\s*[\)\]\}.,;:]")                        # e.g., ") then"
LOWVAR_HEAD   = re.compile(r"^\s*[a-z]\b", re.I)                       # e.g., "i", "r"
ENUM_HEAD_RE  = re.compile(r"^\s*(\(?[ivxlcdmIVXLCDM]+\)|\(?[a-zA-Z]\)|\d+[.)])\s+")  # (i), 1., a)

# NEW: generic detectors for tiny stand-alone parenthetical/micro fragments
PAREN_ONLY    = re.compile(r"^\(\s*[^()]{1,12}\s*\)\.?\s*$")           # e.g., "( f )", "(h′)", "(9.8)"
# Treat as "likely list head" if prev ends with ":" (we avoid merging (i) etc after colons)
ENDS_WITH_COLON = re.compile(r":\s*$")

# NEW (minimal): detect a bare enumeration token like "(ii)", "(b)", "(2)"
ENUM_PAREN_ONLY = re.compile(r"^\(\s*(?:[ivxlcdmIVXLCDM]+|[a-zA-Z]|\d+)\s*\)\.?\s*$")

def _norm_spaces(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def unbalanced_paren_or_quote(s: str) -> bool:
    return (s.count("(") != s.count(")")) or (s.count('"') % 2 != 0) or (s.count("'") % 2 != 0)

def is_parenthetical_fragment(s: str) -> bool:
    """Tiny stand-alone parenthetical like '( f )', '(h′)', '(9.8)'. Keeps variables/equation refs with prior line."""
    return bool(PAREN_ONLY.match(_norm_spaces(s)))

def should_merge(prev: str, curr: str) -> bool:
    """Merge only when the break is clearly structural (paren/math), not normal prose."""
    if not prev or not curr:
        return False
    # --- existing rules ---
    if ENUM_HEAD_RE.search(curr) and not (unbalanced_paren_or_quote(prev) or OP_TAIL_RE.search(prev)):
        return False
    if unbalanced_paren_or_quote(prev):
        return True
    if OP_TAIL_RE.search(prev) and (OP_HEAD_RE.search(curr) or CLOSE_HEAD_RE.search(curr) or LOWVAR_HEAD.search(curr)):
        return True
    if CLOSE_HEAD_RE.search(curr) and not re.search(r"[.?!:]$", prev.strip()):
        return True
    # --- NEW: stitch tiny parenthetical fragment into the previous sentence unless the previous ends with a colon
    # This keeps things like "( f )" attached to "... focal length" while avoiding "(i)" after a list header "The laws are:"
    if is_parenthetical_fragment(curr) and not ENDS_WITH_COLON.search(prev.strip()):
        return True
    return False

def repair_fragments(sentences: List[str]) -> List[str]:
    """Single L→R pass; minimal touch. Never deletes text.
    Minimal tweak: if a standalone '(ii)'/'(b)'/'(2)' line is seen, carry it forward and prefix the next sentence.
    """
    out: List[str] = []
    pending_enum: str = ""  # holds a bare enumeration token to attach to the next sentence

    for s in sentences:
        s = _norm_spaces(s)
        if not s:
            continue

        # If this line is ONLY an enumeration token like "( ii )", don't merge to previous; stash for the next sentence.
        if ENUM_PAREN_ONLY.match(s):
            pending_enum = s
            continue

        # If we have a pending enumeration token, prepend it to the current sentence (the "next" sentence).
        if pending_enum:
            s = f"{pending_enum} {s}"
            pending_enum = ""

        if out and should_merge(out[-1], s):
            out[-1] = _norm_spaces(out[-1] + " " + s)
        else:
            out.append(s)

    # If the very last item was a dangling enumeration token (no next sentence existed), keep it as its own line.
    if pending_enum:
        out.append(pending_enum)

    # if something still unbalanced, greedily attach next until balanced
    i, fixed = 0, []
    while i < len(out):
        cur = out[i]
        if unbalanced_paren_or_quote(cur) and i + 1 < len(out):
            combo = cur + " " + out[i+1]
            i += 2
            while i < len(out) and unbalanced_paren_or_quote(combo):
                combo += " " + out[i]
                i += 1
            fixed.append(_norm_spaces(combo))
        else:
            fixed.append(cur)
            i += 1
    return fixed

# --------- segment retrieved into repaired candidates ---------
sentence_candidates = []   # [{sid, sentence}]
sentence_lookup = {}       # sid -> full entry
sid_counter = 1

for doc in retrieved:
    chunk_uuid = doc.metadata["uuid"]
    page = doc.metadata["page"]
    source = doc.metadata["source"]
    chunk_text = doc.page_content or ""

    raw_sents: List[str] = []
    paragraphs = segmenter.analyze(chunk_text)
    for paragraph in paragraphs:
        for sentence in paragraph:
            sent_text = " ".join(token.value for token in sentence).strip()
            if sent_text:
                raw_sents.append(sent_text)

    fixed_sents = repair_fragments(raw_sents)

    for sentence_text in fixed_sents:
        sid = f"s{sid_counter}"
        sid_counter += 1
        entry = {
            "sid": sid,
            "sentence": sentence_text,
            "page": page,
            "chunk_uuid": chunk_uuid,
            "source": source
        }
        sentence_candidates.append({"sid": sid, "sentence": sentence_text})
        sentence_lookup[sid] = entry

# 🔒 Save chunk-level sentence metadata
with open("sentence_lookup.json", "w", encoding="utf-8") as f:
    json.dump(sentence_lookup, f, indent=2, ensure_ascii=False)

# --------- v1-style minimal LLM selection (unchanged logic) ---------
class SupportIDs(BaseModel):
    ids: List[str]

parser = PydanticOutputParser(pydantic_object=SupportIDs)
format_instructions = parser.get_format_instructions()

prompt = ChatPromptTemplate.from_template("""
You are helping a student understand which textbook sentences support a given answer.

You are given:
- A user question
- An answer to that question
- A list of textbook sentences, each with a short ID

Your task is to identify the **minimum set** of sentence IDs that directly support the answer.

Only include:
- Sentences that clearly explain the final answer
- Definitions, laws, rules, or steps that are explicitly used in the answer
- The answer might be a combination of different related sentences—include those too.
- Ensure to include everything accurately required to completely and clearly answer the student's query.AIMessage
- If any constraints are mentioned, or any point which is important to know for a particular concept include it.
- Unless the query specifies to give only the main point, formula, law, etc. Then no need to include additional ending statements or introductory statements.

Do NOT include:
- General background statements
- Sentences that are only loosely related
- Sentences that repeat the same content with different wording

If multiple sentences say the same thing, prefer the **clearest and most complete** one.
If the same keyword is found in more than one place, use it only if it is clearly required for the answer (don’t keep introductions).

{format_instructions}

---

Question: {query}
Answer: {answer}

Candidate Sentences:
{sentences}
""")

batch_size = 20
num_batches = ceil(len(sentence_candidates) / batch_size)
all_ids = []

for i in range(num_batches):
    batch = sentence_candidates[i * batch_size:(i + 1) * batch_size]
    formatted_sentences = "\n".join(
        f"- [sid: {s['sid']}] {s['sentence']}" for s in batch
    )

    structured_chain = prompt | llm | parser
    try:
        result = structured_chain.invoke({
            "query": query,
            "answer": answer_response.content,   # use final two-step answer
            "sentences": formatted_sentences,
            "format_instructions": format_instructions
        })
        cleaned = []
        for tok in result.ids:
            m = re.search(r"(s\d+)", str(tok))
            if m:
                sid = m.group(1)
                if sid in sentence_lookup:
                    cleaned.append(sid)
        all_ids.extend(cleaned)
    except Exception as e:
        print(f"⚠️ Batch {i+1} failed: {e}")
        continue

# ✅ Final cleanup (document order)
keep = set(all_ids)
returned_sids = [c["sid"] for c in sentence_candidates if c["sid"] in keep]

# ✅ Output
print("\n✅ Supporting Sentence IDs (merged+minimal):\n" + "=" * 50)
print(returned_sids)

print("\n📝 Sentences to Highlight:\n" + "=" * 50)
for sid in returned_sids:
    entry = sentence_lookup.get(sid)
    if entry:
        print(f"{sid} (Page {entry['page']}): {entry['sentence']}")

with open("highlight_ids.json", "w", encoding="utf-8") as f:
    json.dump(returned_sids, f, indent=2)



✅ Supporting Sentence IDs (merged+minimal):
['s6', 's8', 's9', 's11', 's12', 's13', 's14', 's15', 's16', 's19', 's20', 's29', 's30', 's31', 's32']

📝 Sentences to Highlight:
s6 (Page 15): The value of the refractive index for a given pair of media depends upon the speed of light in the two media , as given below .
s8 (Page 15): Let v1 be the speed of light in medium 1 and v2 be the speed of light in medium 2 .
s9 (Page 15): The refractive index of medium 2 with respect to medium 1 is given by the ratio of the speed of light in medium 1 and the speed of light in medium 2 .
s11 (Page 15): This can be expressed in an equation form as n21 = Speed of light in medium 1 Speed of light in medium 2 = v v 1 2 ( 9.5 )
s12 (Page 15): By the same argument , the refractive index of medium 1 with respect to medium 2 is represented as n12 .
s13 (Page 15): It is given by n12 = Speed of light in medium 2 Speed of light in medium 1 = v v 2 1 ( 9.6 )
s14 (Page 15): If medium 1 is vacuum or air , then the

In [197]:
# # CELL 4 — coverage-aware selection from Cell 3 + safe cleaning (verbatim)
# import json, re
# from pathlib import Path
# from typing import List
# from pydantic import BaseModel
# from langchain.prompts import ChatPromptTemplate
# from langchain.output_parsers import PydanticOutputParser

# base = Path(".")
# sel_ids_path = base / "highlight_ids.json"       # from Cell 3
# lookup_path  = base / "sentence_lookup.json"     # from Cell 3
# clean_out    = base / "ai_cleaned_sentences.json"

# # ---- load Cell 3 outputs
# with open(sel_ids_path, "r", encoding="utf-8") as f:
#     selected_ids = json.load(f)

# with open(lookup_path, "r", encoding="utf-8") as f:
#     sentence_lookup = json.load(f)

# # keep original order coming from Cell 3
# candidates = []
# for sid in selected_ids:
#     ent = sentence_lookup.get(sid)
#     if not ent: 
#         continue
#     candidates.append({
#         "sid": sid,
#         "page": int(ent["page"]),
#         "sentence": ent["sentence"].strip()
#     })

# # ===== PASS 1: SELECT IDS THAT COVER THE FINAL ANSWER =====

# class KeepIDs(BaseModel):
#     ids: List[str]  # subset of provided sids, in doc order

# select_parser = PydanticOutputParser(pydantic_object=KeepIDs)
# select_instr = select_parser.get_format_instructions()

# select_prompt = ChatPromptTemplate.from_template("""
# You will choose the MINIMUM set of candidate sentences that collectively support the FINAL ANSWER.

# Inputs:
# - QUESTION: {query}
# - FINAL ANSWER: {answer}

# CANDIDATES (in document order):
# {cands}

# Selection rules:
# - Pick ONLY from the listed IDs. Preserve the same order.
# - Cover EVERY major claim in the Final Answer at least once (definition/identity, extent/locations, subdivisions or lists,
#   causes/processes, consequences, examples, etc.).
# - Prefer clear, specific sentences that directly *state* the claim used in the answer.
# - Exclude headings, labels, page furniture, and fragmentary tokens (e.g., "( b ) Himalayas", single words, things ending with ":").
# - For short answers (~≤120 words): keep ~2–6 sentences.
#   For long answers (~≥150–300 words): keep ~6–12 sentences.
# - NEVER drop a sentence if it is the ONLY one that states a kept claim.

# Return ONLY this JSON:
# {format_instructions}
# """)

# def render_cands(cands):
#     return "\n".join(f"- [sid: {c['sid']}] {c['sentence']}" for c in cands)

# select_chain = select_prompt | llm | select_parser
# keep = select_chain.invoke({
#     "query": query,
#     "answer": answer_response.content,
#     "cands": render_cands(candidates),
#     "format_instructions": select_instr
# }).ids

# # keep IDs in the original order & intersect with available
# keep_set = {sid for sid in keep if sid in {c["sid"] for c in candidates}}
# kept = [c for c in candidates if c["sid"] in keep_set]

# # ===== PASS 2: CLEAN THE KEPT SENTENCES SAFELY (NO PROSE FRACTURE) =====

# # heuristics
# BULLET_JOIN_RE = re.compile(r"(?:\s[•●▪‣■]\s|\sz\s)", re.I)  # split only when bullets are truly joined
# NUM_LIST_RE    = re.compile(r"\b(?:\(?[0-9]+|[ivxlcdm]+)\)?[.)]\s+\S+", re.I)
# TAIL_RE = re.compile(
#     r"(?i)\b(observe|write|group|classify|find out|according to|answer|project|activity|exercise|assignment|"
#     r"try this|can you|let us|discuss|for example|figure|fig\.|table|chart)\b"
# )

# def looks_joined(s: str) -> bool:
#     return bool(BULLET_JOIN_RE.search(s)) or (len(re.findall(NUM_LIST_RE, s)) >= 2)

# def trim_tail(s: str) -> str:
#     m = TAIL_RE.search(s)
#     return s[:m.start()].rstrip(" .,:;–—-") if m else s

# def split_joined(s: str) -> List[str]:
#     # turn ' z ' into ' • ' split marker without touching 'Brazil'
#     t = re.sub(r"\s+", " ", s).strip()
#     t = re.sub(r"^(?:z)\s+", "• ", t)    # start
#     t = re.sub(r"\s(z)\s", " • ", t)     # middle
#     parts = [p.strip(" •-–—") for p in t.split(" • ")]
#     parts = [p for p in parts if len(p.split()) >= 2]
#     return parts if len(parts) >= 2 else [s.strip()]

# def strip_leading_markers(s: str) -> str:
#     s = re.sub(r"^\s*[•●▪‣■]\s*", "", s)                        # symbol bullet
#     s = re.sub(r"^\s*(?:\(?[0-9]+|[ivxlcdm]+)\)?[.)-]\s*", "", s, flags=re.I)  # 1.  (i)  2)
#     return s.strip()

# def present_in_original(orig: str, piece: str) -> bool:
#     if piece in orig: return True
#     def norm(x: str):
#         x = re.sub(r"\s*\(\s*", " (", x)
#         x = re.sub(r"\s*\)\s*", ")", x)
#         x = re.sub(r"\s+", " ", x)
#         return x.strip()
#     return norm(piece) in norm(orig)

# class CleanItem(BaseModel):
#     sid: str
#     pieces: List[str]  # verbatim substrings after allowed trims (order preserved)

# class CleanBatch(BaseModel):
#     items: List[CleanItem]

# clean_parser = PydanticOutputParser(pydantic_object=CleanBatch)
# clean_instr = clean_parser.get_format_instructions()

# clean_prompt = ChatPromptTemplate.from_template("""
# You will prepare the kept sentences for highlighting EXACTLY as they appear.

# For each entry:
# - If it is a normal prose sentence, return ONE piece (verbatim), only trimming an instruction tail if present.
# - If it is a run-on bullet line (joined by "•" / "z" / numbered items), split into separate pieces (order preserved).
# - Remove ONLY leading bullet/number markers from each piece.
# - Keep parentheses that belong to the phrase.
# - Do NOT rephrase or shorten normal sentences.
# - Ignore headings/labels (e.g., items that are just a letter/number or end with ":").

# Return ONLY this JSON:
# {format_instructions}

# Entries:
# {entries}
# """)

# # Build small cleaning batches with only kept items
# BATCH = 16
# kept_batches = [kept[i:i+BATCH] for i in range(0, len(kept), BATCH)]
# clean_results = []  # [{"sid","page","pieces":[...]}]

# for group in kept_batches:
#     entries_text = "\n".join(f"- [sid: {e['sid']}] {e['sentence']}" for e in group)
#     chain = clean_prompt | llm | clean_parser
#     out = chain.invoke({
#         "entries": entries_text,
#         "format_instructions": clean_instr
#     })

#     # guard-rails: verify verbatim & fall back for prose
#     page_map = {e["sid"]: e["page"] for e in group}
#     orig_map = {e["sid"]: e["sentence"] for e in group}

#     for item in out.items:
#         sid = item.sid
#         orig = orig_map.get(sid, "")
#         page = page_map.get(sid, 1)

#         # decide prose vs joined using the original text (not AI output)
#         prose = not looks_joined(orig)

#         if prose:
#             # one piece: trim tail; if AI gave >1 or tiny fragments, fall back to full trimmed sentence
#             cand = item.pieces[0] if item.pieces else orig
#             cand = strip_leading_markers(trim_tail(cand)).strip() or strip_leading_markers(trim_tail(orig))
#             # minimal presence/length check
#             if not present_in_original(orig, cand) or len(cand.split()) < max(3, int(0.4 * len(orig.split()))):
#                 cand = strip_leading_markers(trim_tail(orig))
#             clean_results.append({"sid": sid, "page": page, "pieces": [cand]})
#         else:
#             # run-on bullets: split locally too (safety), then verify each piece
#             ai_pieces = item.pieces if item.pieces else split_joined(orig)
#             pieces = []
#             for p in ai_pieces:
#                 p2 = strip_leading_markers(trim_tail(p)).strip()
#                 if p2 and present_in_original(orig, p2):
#                     pieces.append(p2)
#             if not pieces:
#                 pieces = [strip_leading_markers(trim_tail(orig))]
#             clean_results.append({"sid": sid, "page": page, "pieces": pieces})

# # ---- write
# with open(clean_out, "w", encoding="utf-8") as f:
#     json.dump(clean_results, f, indent=2, ensure_ascii=False)

# print(f"✅ Kept {len(kept)} ids; prepared {sum(len(r['pieces']) for r in clean_results)} highlight pieces → {clean_out.name}")


In [198]:
# CELL 4 — coverage-aware selection + safe cleaning (balanced refinement)
import json, re
from pathlib import Path
from typing import List
from pydantic import BaseModel
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

base = Path(".")
sel_ids_path = base / "highlight_ids.json"       # from Cell 3
lookup_path  = base / "sentence_lookup.json"     # from Cell 3
clean_out    = base / "ai_cleaned_sentences.json"

# ---- load Cell 3 outputs
with open(sel_ids_path, "r", encoding="utf-8") as f:
    selected_ids = json.load(f)

with open(lookup_path, "r", encoding="utf-8") as f:
    sentence_lookup = json.load(f)

candidates = []
allowed = set(selected_ids)
for sid in selected_ids:
    ent = sentence_lookup.get(sid)
    if not ent:
        continue
    candidates.append({
        "sid": sid,
        "page": int(ent["page"]),
        "sentence": (ent["sentence"] or "").strip()
    })

# Get the final answer text
try:
    answer_text = answer
except NameError:
    try:
        answer_text = answer_response.content
    except NameError:
        try:
            answer_text = final_answer_text
        except NameError:
            answer_text = ""

# ---- intent phrase (neutral unless the query clearly asks for relative or absolute) ----
def _intent_phrase(q: str, a: str) -> str:
    s = f"{q or ''} {a or ''}".lower()
    if re.search(r"\b(between\s+(two|2)\s+(media|mediums)|relative refractive|with respect to|n\s*21|n\s*12|v1|v2)\b", s):
        return "between two media → favor relative forms (n21 = v1/v2 and n12 = v2/v1); keep both reciprocals"
    if re.search(r"\b(absolute|vacuum|air|c\s*/\s*v|nm\b)\b", s):
        return "with respect to vacuum/air → include absolute form (nm = c/v) when relevant"
    return "neutral (no preference)"

# ===== PASS 1: CONTEXT-AWARE SELECTION =====
class KeepIDs(BaseModel):
    ids: List[str]

select_parser = PydanticOutputParser(pydantic_object=KeepIDs)
select_instr = select_parser.get_format_instructions()

def render_cands(cands):
    return "\n".join(f"- [sid: {c['sid']}] (page {c['page']}) {c['sentence']}" for c in cands)

kept = []
if candidates:
    select_prompt = ChatPromptTemplate.from_template("""
    You will choose the set of candidate sentences that best support the FINAL ANSWER.

    Inputs:
    - QUESTION: {query}
    - FINAL ANSWER: {answer}
    - INTENT: {intent_phrase}

    CANDIDATES (document order, with page numbers):
    {cands}

    Selection rules:
    - Keep ALL sentences that directly support or clarify the Final Answer.
    - INCLUDE useful introductory context if it frames numbered laws/definitions (e.g., “The following are the laws of …”).
    - Prefer canonical formulations (laws/definitions/rules/equations).
    - Read the INTENT:
      • If it says "between two media", favor **relative refractive index** statements and **keep both reciprocal forms** (n21 = v1/v2 and n12 = v2/v1).
      • If it says "with respect to vacuum/air", allow **absolute refractive index** (nm = c/v) when relevant.
      • If it says "neutral", apply no preference.
    - Remove only:
      • Exact duplicates of the same sentence (from chunking)
      • Obvious page furniture (headers/footers like “Science”, “Figure…”, “Table…”)
      • Irrelevant reminders like “Let us recall…”, “Remember that…”
    - Do NOT remove sentences just to reduce the count.
    - Preserve order.

    Return ONLY this JSON:
    {format_instructions}
    """)

    try:
        keep_ids = select_prompt | llm | select_parser
        keep_ids = keep_ids.invoke({
            "query": query,
            "answer": answer_text,
            "cands": render_cands(candidates),
            "format_instructions": select_instr,
            "intent_phrase": _intent_phrase(query, answer_text),
        }).ids
    except Exception:
        keep_ids = []

    keep_set = set(i for i in keep_ids if i in allowed)
    kept = [c for c in candidates if c["sid"] in keep_set]

# ===== Fallback if LLM fails =====
if not kept:
    def _norm_tokens(s: str):
        s = re.sub(r"[^\w\s]", " ", s.lower())
        return set(w for w in s.split() if len(w) > 2)

    INTRO_PAT = re.compile(r"^(let us recall|remember that|you are already familiar)\b", re.I)
    FURNITURE_PAT = re.compile(r"\b(science|figure|fig\.|table|chart|exercise|activity)\b", re.I)

    ans_toks = _norm_tokens(answer_text)
    scored = []
    for c in candidates:
        if INTRO_PAT.search(c["sentence"]) or FURNITURE_PAT.search(c["sentence"]):
            continue
        toks = _norm_tokens(c["sentence"])
        overlap = len(ans_toks & toks)
        scored.append((overlap, c["page"], c))
    scored.sort(key=lambda x: (-x[0], x[1]))
    kept = [t[2] for t in scored[:max(6, len(candidates)//2)]] or candidates[:2]

# ===== PASS 2: CLEANING (unchanged) =====
BULLET_JOIN_RE = re.compile(r"(?:\s[•●▪‣■]\s|\sz\s)", re.I)
NUM_LIST_RE    = re.compile(r"\b(?:\(?[0-9]+|[ivxlcdm]+)\)?[.)]\s+\S+", re.I)
TAIL_RE = re.compile(
    r"(?i)\b(observe|write|group|classify|find out|according to|answer|project|exercise|assignment|"
    r"try this|can you|let us|discuss|for example|figure|fig\.|table|chart|science)\b"
)

def looks_joined(s: str) -> bool:
    return bool(BULLET_JOIN_RE.search(s)) or (len(re.findall(NUM_LIST_RE, s)) >= 2)

def trim_tail(s: str) -> str:
    m = TAIL_RE.search(s)
    return s[:m.start()].rstrip(" .,:;–—-") if m else s

def split_joined(s: str) -> List[str]:
    t = re.sub(r"\s+", " ", s).strip()
    t = re.sub(r"^(?:z)\s+", "• ", t)
    t = re.sub(r"\s(z)\s", " • ", t)
    parts = [p.strip(" •-–—") for p in t.split(" • ")]
    parts = [p for p in parts if len(p.split()) >= 2]
    return parts if len(parts) >= 2 else [s.strip()]

def strip_leading_markers(s: str) -> str:
    s = re.sub(r"^\s*[•●▪‣■]\s*", "", s)
    s = re.sub(r"^\s*(?:\(?[0-9]+|[ivxlcdm]+)\)?[.)-]\s*", "", s, flags=re.I)
    return s.strip()

def present_in_original(orig: str, piece: str) -> bool:
    if piece in orig: return True
    def norm(x: str):
        x = re.sub(r"\s*\(\s*", " (", x)
        x = re.sub(r"\s*\)\s*", ")", x)
        x = re.sub(r"\s+", " ", x)
        return x.strip()
    return norm(piece) in norm(orig)

class CleanItem(BaseModel):
    sid: str
    pieces: List[str]

class CleanBatch(BaseModel):
    items: List[CleanItem]

clean_parser = PydanticOutputParser(pydantic_object=CleanBatch)
clean_instr = clean_parser.get_format_instructions()

clean_prompt = ChatPromptTemplate.from_template("""
You will prepare the kept sentences for highlighting EXACTLY as they appear.

For each entry:
- If it is a normal prose sentence, return ONE piece (verbatim), only trimming an instruction tail if present.
- If it is a run-on bullet line (joined by "•" / "z" / numbered items), split into separate pieces (order preserved).
- Remove ONLY leading bullet/number markers from each piece.
- Keep parentheses that belong to the phrase.
- Do NOT rephrase or shorten normal sentences.
- Ignore headings/labels or page furniture (e.g., "Science", "Exercise", "Figure", "Table").
Return ONLY this JSON:
{format_instructions}

Entries:
{entries}
""")

BATCH = 16
kept_batches = [kept[i:i+BATCH] for i in range(0, len(kept), BATCH)]
clean_results = []

for group in kept_batches:
    entries_text = "\n".join(f"- [sid: {e['sid']}] {e['sentence']}" for e in group)
    chain = clean_prompt | llm | clean_parser
    out = chain.invoke({
        "entries": entries_text,
        "format_instructions": clean_instr
    })

    page_map = {e["sid"]: e["page"] for e in group}
    orig_map = {e["sid"]: e["sentence"] for e in group}

    for item in out.items:
        sid = item.sid
        orig = orig_map.get(sid, "")
        page = page_map.get(sid, 1)

        prose = not looks_joined(orig)

        if prose:
            cand = item.pieces[0] if item.pieces else orig
            cand = strip_leading_markers(trim_tail(cand)).strip() or strip_leading_markers(trim_tail(orig))
            if not present_in_original(orig, cand) or len(cand.split()) < max(3, int(0.4 * len(orig.split()))):
                cand = strip_leading_markers(trim_tail(orig))
            clean_results.append({"sid": sid, "page": page, "pieces": [cand]})
        else:
            ai_pieces = item.pieces if item.pieces else split_joined(orig)
            pieces = []
            for p in ai_pieces:
                p2 = strip_leading_markers(trim_tail(p)).strip()
                if p2 and present_in_original(orig, p2):
                    pieces.append(p2)
            if not pieces:
                pieces = [strip_leading_markers(trim_tail(orig))]
            clean_results.append({"sid": sid, "page": page, "pieces": pieces})

# ---- write
with open(clean_out, "w", encoding="utf-8") as f:
    json.dump(clean_results, f, indent=2, ensure_ascii=False)

print(f"✅ Kept {len(kept)} ids; prepared {sum(len(r['pieces']) for r in clean_results)} highlight pieces → {clean_out.name}")


✅ Kept 11 ids; prepared 11 highlight pieces → ai_cleaned_sentences.json


In [199]:
# # CELL 5 — highlight using sliding-window matcher with hyphen-aware tokenization
# import fitz  # PyMuPDF
# import json
# import re
# import unicodedata
# from pathlib import Path
# from collections import defaultdict

# # 📁 Paths
# base_dir = Path(".")
# pdf_in = base_dir / "geography.pdf"
# pdf_out = base_dir / "highlighted_output.pdf"

# # Primary input: AI-cleaned pieces from Cell 4
# clean_in = base_dir / "ai_cleaned_sentences.json"

# # Fallback (if you skip Cell 4): original ids + lookup from Cell 3
# ids_path = base_dir / "highlight_ids.json"
# lookup_path = base_dir / "sentence_lookup.json"

# # 🔤 Hyphen/ligature/space normalization → tokens
# HYPHENS = r"[\-\u2010-\u2015\u2212]"  # -, ‐ - ‒ – — −
# def norm_tokenize(s: str):
#     # NFKC to unify glyphs; drop soft hyphen; turn hyphens into spaces (so "east-flowing" -> "east flowing")
#     s = unicodedata.normalize("NFKC", s).replace("\u00ad", "")
#     s = re.sub(HYPHENS, " ", s)
#     s = re.sub(r"[^\w\s]", " ", s)
#     s = re.sub(r"\s+", " ", s)
#     s = s.strip().lower()
#     return s.split() if s else []

# # Parentheses variants for fallback search
# def paren_variants(s: str):
#     v1 = s
#     v2 = re.sub(r"\s*\(\s*", " (", v1)
#     v2 = re.sub(r"\s*\)\s*", ")", v2)
#     v3 = re.sub(r"[()]", "", v2)
#     v3 = re.sub(r"\s+", " ", v3).strip()
#     return [v1, v2, v3]

# # Load inputs
# targets = []  # {"page": int, "text": str}
# if clean_in.exists():
#     data = json.loads(clean_in.read_text(encoding="utf-8"))
#     for row in data:
#         page = int(row["page"])
#         for piece in row.get("pieces", []):
#             piece = piece.strip()
#             if piece:
#                 targets.append({"page": page, "text": piece})
# else:
#     highlight_ids = json.loads(ids_path.read_text(encoding="utf-8"))
#     sentence_lookup = json.loads(lookup_path.read_text(encoding="utf-8"))
#     for sid in highlight_ids:
#         entry = sentence_lookup.get(sid)
#         if entry:
#             targets.append({"page": int(entry["page"]), "text": entry["sentence"]})

# # Group per page
# by_page = defaultdict(list)
# for t in targets:
#     by_page[int(t["page"])].append(t["text"])

# doc = fitz.open(pdf_in)
# total_spans, misses = 0, []

# for page_num, pieces in sorted(by_page.items()):
#     page = doc[page_num - 1]

#     # words: [x0, y0, x1, y1, "word", block_no, line_no, word_no]  (older builds may have only first 5)
#     words = page.get_text("words")
#     has_blocks = len(words[0]) >= 8 if words else False

#     # Build block -> word list
#     blocks = defaultdict(list)
#     if has_blocks:
#         for w in words:
#             blocks[w[5]].append(w)
#     else:
#         blocks[-1] = words

#     for piece in pieces:
#         target_tokens = norm_tokenize(piece)
#         if not target_tokens:
#             continue

#         found = False

#         # Try block-scoped sliding-window with hyphen-aware tokenization
#         for b_idx, wlist in blocks.items():
#             if not wlist:
#                 continue

#             # Build token stream with a map back to word indices
#             tok_stream = []     # list of tokens
#             tok2word   = []     # parallel list of word indices
#             for wi, w in enumerate(wlist):
#                 w_tokens = norm_tokenize(w[4])  # may yield multiple tokens if word had hyphen
#                 for _t in w_tokens:
#                     tok_stream.append(_t)
#                     tok2word.append(wi)

#             n, m = len(tok_stream), len(target_tokens)
#             if m == 0 or n < m:
#                 continue

#             for i in range(n - m + 1):
#                 if tok_stream[i:i+m] == target_tokens:
#                     # Map token span back to word rectangles (unique word indices in order)
#                     word_idx_span = sorted(set(tok2word[i:i+m]))
#                     rects = []
#                     for idx in word_idx_span:
#                         x0, y0, x1, y1 = wlist[idx][0:4]
#                         rects.append(fitz.Rect(x0, y0, x1, y1))
#                     for r in rects:
#                         page.add_highlight_annot(r)
#                         total_spans += 1
#                     found = True
#                     break
#             if found:
#                 break

#         if not found:
#             # Fallback: page.search_for with parentheses & hyphen variants
#             # Try with original, then with hyphens normalized to spaces
#             tried = False
#             for variant in paren_variants(piece):
#                 # replace hyphens with spaces in search text for safety
#                 v = unicodedata.normalize("NFKC", variant).replace("\u00ad", "")
#                 v = re.sub(HYPHENS, " ", v)
#                 v = re.sub(r"\s+", " ", v).strip()
#                 if not v:
#                     continue
#                 quads = page.search_for(v, quads=True)
#                 tried = True
#                 if quads:
#                     for q in quads:
#                         page.add_highlight_annot(q)
#                         total_spans += 1
#                     found = True
#                     break

#             if not found:
#                 misses.append(piece[:90])

# doc.save(pdf_out, garbage=4, deflate=True)
# doc.close()

# print(f"✅ Highlighted {total_spans} spans → {pdf_out}")
# if misses:
#     print("⚠️ Not found (sample):", misses[:6])


In [ ]:
# # CELL 5 — robust highlighter (hyphen/quote/paren aware + tolerant fallback)
# import fitz  # PyMuPDF
# import json
# import re
# import unicodedata
# from math import ceil
# from pathlib import Path
# from collections import defaultdict

# # ---------- paths
# base_dir = Path(".")
# # pdf_in   = base_dir / "geography.pdf"
# # pdf_in   = base_dir / "biology.pdf"
# # pdf_in   = base_dir / "class10phy.pdf"
# # pdf_in   = base_dir / "history.pdf"

# pdf_out  = base_dir / "highlighted_output.pdf"
# clean_in = base_dir / "ai_cleaned_sentences.json"     # from Cell 4
# ids_path = base_dir / "highlight_ids.json"            # fallback if Cell 4 is skipped
# lookup_path = base_dir / "sentence_lookup.json"       # fallback

# # ---------- normalization / tokenization

# HYPHENS = r"[\-\u2010-\u2015\u2212]"  # -, ‐, -, ‒, –, —, −
# QUOTES  = "[\u2018\u2019\u201A\u2032\u2035\u201C\u201D\u201E\u2033\u2036']"

# def fold_diacritics(s: str) -> str:
#     # keep base letters; strip combining marks
#     s = unicodedata.normalize("NFD", s)
#     s = "".join(c for c in s if not unicodedata.combining(c))
#     return unicodedata.normalize("NFC", s)

# def norm_tokenize(s: str):
#     # unify glyphs, strip soft hyphen, treat hyphens/dashes as spaces, normalize quotes to spaces
#     s = unicodedata.normalize("NFKC", s).replace("\u00ad", "")
#     s = fold_diacritics(s)
#     s = re.sub(HYPHENS, " ", s)
#     s = re.sub(QUOTES, " ", s)
#     s = re.sub(r"[^\w\s]", " ", s)        # drop remaining punctuation
#     s = re.sub(r"\s+", " ", s).strip().lower()
#     return s.split() if s else []

# def paren_variants(s: str):
#     v1 = s
#     v2 = re.sub(r"\s*\(\s*", " (", v1)
#     v2 = re.sub(r"\s*\)\s*", ")", v2)
#     v3 = re.sub(r"[()]", "", v2)
#     v3 = re.sub(r"\s+", " ", v3).strip()
#     return [v1, v2, v3]

# # ---------- utilities

# def build_blocks(page):
#     """Return {block_idx: [word tuples]} using page.get_text('words')."""
#     words = page.get_text("words")  # (x0,y0,x1,y1,text,block_no,line_no,word_no) on most builds
#     if not words:
#         return {-1: []}
#     has_blocks = len(words[0]) >= 8
#     blocks = defaultdict(list)
#     if has_blocks:
#         for w in words:
#             blocks[w[5]].append(w)
#     else:
#         blocks[-1] = words
#     return blocks

# def stream_from_words(wlist):
#     """Token stream from a block's word list, mapping back to word indices."""
#     stream, tok2word = [], []
#     for wi, w in enumerate(wlist):
#         toks = norm_tokenize(w[4])  # hyphen-aware; might yield multiple tokens
#         for t in toks:
#             stream.append(t)
#             tok2word.append(wi)
#     return stream, tok2word

# def rects_from_span(wlist, tok2word, i0, i1):
#     """Rectangles covering unique word indices in token window [i0, i1] inclusive."""
#     word_idx_span = sorted(set(tok2word[i0:i1+1]))
#     return [fitz.Rect(*wlist[idx][0:4]) for idx in word_idx_span]

# def exact_window(stream, target):
#     """Exact contiguous match. Returns (i0, i1) or None."""
#     n, m = len(stream), len(target)
#     if m == 0 or n < m:
#         return None
#     for i in range(n - m + 1):
#         if stream[i:i+m] == target:
#             return (i, i+m-1)
#     return None

# def tolerant_window(stream, target, max_miss=1, min_ratio=0.9):
#     """Allow a small # of token mismatches for long targets (>= 10 tokens)."""
#     n, m = len(stream), len(target)
#     if m == 0 or n < m:
#         return None
#     need = max(int(ceil(min_ratio * m)), m - max_miss)
#     best = None
#     for i in range(n - m + 1):
#         win = stream[i:i+m]
#         matches = sum(1 for a, b in zip(win, target) if a == b)
#         if matches >= need:
#             # tighten to left/right actual match extent (approx): choose contiguous hits
#             # simple approach: use the full window mapping; it's fine for highlighting
#             best = (i, i+m-1)
#             break
#     return best

# # ---------- load targets

# targets = []  # {"page": int, "text": str}

# if clean_in.exists():
#     data = json.loads(clean_in.read_text(encoding="utf-8"))
#     for row in data:
#         page = int(row["page"])
#         for piece in row.get("pieces", []):
#             piece = piece.strip()
#             if piece:
#                 targets.append({"page": page, "text": piece})
# else:
#     # fallback to raw ids
#     ids = json.loads(ids_path.read_text(encoding="utf-8"))
#     lookup = json.loads(lookup_path.read_text(encoding="utf-8"))
#     for sid in ids:
#         ent = lookup.get(sid)
#         if ent:
#             targets.append({"page": int(ent["page"]), "text": ent["sentence"]})

# # group targets per page
# by_page = defaultdict(list)
# for t in targets:
#     by_page[int(t["page"])].append(t["text"])

# # ---------- highlight

# doc = fitz.open(pdf_in)
# total_spans, misses = 0, []

# for page_num, pieces in sorted(by_page.items()):
#     page = doc[page_num - 1]
#     blocks = build_blocks(page)

#     for raw in pieces:
#         found = False
#         target_tokens = norm_tokenize(raw)
#         if not target_tokens:
#             continue

#         # 1) block-scoped exact token window
#         for b_idx, wlist in blocks.items():
#             stream, tok2word = stream_from_words(wlist)
#             loc = exact_window(stream, target_tokens)
#             if loc:
#                 rlist = rects_from_span(wlist, tok2word, *loc)
#                 for r in rlist:
#                     page.add_highlight_annot(r)
#                     total_spans += 1
#                 found = True
#                 break
#         if found:
#             continue

#         # 2) page-wide exact token window
#         all_words = [w for bl in blocks.values() for w in bl]
#         stream, tok2word = stream_from_words(all_words)
#         loc = exact_window(stream, target_tokens)
#         if loc:
#             rlist = rects_from_span(all_words, tok2word, *loc)
#             for r in rlist:
#                 page.add_highlight_annot(r)
#                 total_spans += 1
#             continue

#         # 3) search_for with paren/quote/hyphen variants
#         tried = False
#         for variant in paren_variants(raw):
#             v = unicodedata.normalize("NFKC", variant).replace("\u00ad", "")
#             v = fold_diacritics(v)
#             v = re.sub(HYPHENS, " ", v)
#             v = re.sub(QUOTES, " ", v)
#             v = re.sub(r"\s+", " ", v).strip()
#             if not v:
#                 continue
#             tried = True
#             quads = page.search_for(v, quads=True)
#             if quads:
#                 for q in quads:
#                     page.add_highlight_annot(q)
#                     total_spans += 1
#                 found = True
#                 break
#         if found:
#             continue

#         # 4) tolerant page-wide token window (only for longer phrases)
#         if len(target_tokens) >= 10:
#             loc = tolerant_window(stream, target_tokens, max_miss=1, min_ratio=0.9)
#             if loc:
#                 rlist = rects_from_span(all_words, tok2word, *loc)
#                 for r in rlist:
#                     page.add_highlight_annot(r)
#                     total_spans += 1
#                 found = True

#         if not found:
#             misses.append(raw[:90])

# doc.save(pdf_out, garbage=4, deflate=True)
# doc.close()

# print(f"✅ Highlighted {total_spans} spans → {pdf_out}")
# if misses:
#     print("⚠️ Not found (sample):", misses[:6])


✅ Highlighted 249 spans → highlighted_output.pdf


In [208]:
# CELL 5 — robust highlighter (token-aware + tolerant + scoped symbol highlights)
import fitz  # PyMuPDF
import json
import re
import unicodedata
from math import ceil
from pathlib import Path
from collections import defaultdict

# ---------- paths
base_dir = Path(".")
# pdf_in   = base_dir / "geography.pdf"
# pdf_in   = base_dir / "biology.pdf"
pdf_in   = base_dir / "class10phy.pdf"
# pdf_in   = base_dir / "history.pdf"

pdf_out  = base_dir / "highlighted_output.pdf"
clean_in = base_dir / "ai_cleaned_sentences.json"     # from Cell 4
ids_path = base_dir / "highlight_ids.json"            # fallback if Cell 4 is skipped
lookup_path = base_dir / "sentence_lookup.json"       # fallback

# ---------- normalization / tokenization

# Hyphens to normalize to spaces
HYPHENS = r"[\-\u2010-\u2015\u2212]"  # -, ‐, –, —, −

# IMPORTANT: do NOT include Unicode primes here; we want to preserve them.
# Keep only typographic quotes/apostrophes.
QUOTES  = r"[\u2018\u2019\u201A\u201C\u201D\u201E']"

# Unicode prime characters (′ ″ ‴ ⁗)
PRIME_CHARS = "\u2032\u2033\u2034\u2057"

def fold_diacritics(s: str) -> str:
    s = unicodedata.normalize("NFD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    return unicodedata.normalize("NFC", s)

def normalize_primes_to_tokens(s: str) -> str:
    # Map primes to textual tokens so token windows align even when PDF splits them.
    # e.g., "h′" → "h prime", "″" → " doubleprime"
    s = s.replace("\u2032", " prime")
    s = s.replace("\u2033", " doubleprime")
    s = s.replace("\u2034", " tripleprime")
    s = s.replace("\u2057", " quadrupleprime")
    return s

def prime_variants(s: str):
    # Return variants that keep primes intact or map them to ASCII apostrophes / remove.
    # Used by search_for to increase robustness.
    v = s
    # 1) raw (keep exact)
    yield v
    # 2) map primes to ASCII apostrophe
    v2 = v.translate({ord("\u2032"): ord("'"), ord("\u2033"): ord('"')})
    yield v2
    # 3) drop primes (sometimes PDF glues them to letters awkwardly)
    v3 = re.sub(f"[{PRIME_CHARS}]", "", v)
    yield v3

def norm_tokenize(s: str):
    # tokenization that preserves semantic symbols:
    # - keep primes by mapping them to tokens (prime/doubleprime/…)
    # - keep slashes for ratios via later operator highlighting; for token stream, we drop punctuation
    s = unicodedata.normalize("NFKC", s).replace("\u00ad", "")
    s = fold_diacritics(s)
    s = normalize_primes_to_tokens(s)            # <-- preserve as words
    s = re.sub(HYPHENS, " ", s)
    s = re.sub(QUOTES, " ", s)
    # Keep all other punctuation out of tokens
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip().lower()
    return s.split() if s else []

def paren_variants(s: str):
    # Variants for parentheses spacing; keep primes intact
    v1 = s
    v2 = re.sub(r"\s*\(\s*", " (", v1)
    v2 = re.sub(r"\s*\)\s*", ")", v2)
    v3 = re.sub(r"[()]", "", v2)
    v3 = re.sub(r"\s+", " ", v3).strip()
    return [v1, v2, v3]

# ---------- operator/symbol support (scoped to matched sentence region)
OP_CHARS = (
    "=<>±≈≠≤≥∝↔→"          # relations
    "+−-*/^×·÷"            # arithmetic (ASCII '-' and U+2212 '−')
    "√∞∑Σ∫πθΔ∂µΩ°"
)

# Also treat parentheses and primes as micro-highlights inside region
EXTRA_REGION_SYMS = "()"
PRIME_SET = set(PRIME_CHARS)

OP_STRINGS = ["<=", ">=", "==", "!=", "+=", "-=", "->", "<-", "<->"]

def has_ops_or_primes_or_parens(s: str) -> bool:
    return (
        any(ch in s for ch in OP_CHARS)
        or any(seq in s for seq in OP_STRINGS)
        or any(ch in s for ch in EXTRA_REGION_SYMS)
        or any(ch in s for ch in PRIME_SET)
    )

# ---------- utilities
def build_blocks(page):
    """Return {block_idx: [word tuples]} using page.get_text('words')."""
    words = page.get_text("words")  # (x0,y0,x1,y1,text,block_no,line_no,word_no)
    if not words:
        return {-1: []}
    has_blocks = len(words[0]) >= 8
    blocks = defaultdict(list)
    if has_blocks:
        for w in words:
            blocks[w[5]].append(w)
    else:
        blocks[-1] = words
    return blocks

def stream_from_words(wlist):
    stream, tok2word = [], []
    for wi, w in enumerate(wlist):
        toks = norm_tokenize(w[4])    # tokenizing PDF word text with same rules
        for t in toks:
            stream.append(t)
            tok2word.append(wi)
    return stream, tok2word

def rects_from_span(wlist, tok2word, i0, i1):
    idxs = sorted(set(tok2word[i0:i1+1]))
    return [fitz.Rect(*wlist[idx][0:4]) for idx in idxs]

def exact_window(stream, target):
    n, m = len(stream), len(target)
    if m == 0 or n < m:
        return None
    for i in range(n - m + 1):
        if stream[i:i+m] == target:
            return (i, i+m-1)
    return None

def tolerant_window(stream, target, max_miss=1, min_ratio=0.9):
    n, m = len(stream), len(target)
    if m == 0 or n < m:
        return None
    need = max(int(ceil(min_ratio * m)), m - max_miss)
    for i in range(n - m + 1):
        win = stream[i:i+m]
        matches = sum(1 for a, b in zip(win, target) if a == b)
        if matches >= need:
            return (i, i+m-1)
    return None

def union_rect(rects):
    if not rects:
        return None
    u = fitz.Rect(rects[0])
    for r in rects[1:]:
        u |= r
    return u

# ---------- load targets
targets = []  # {"page": int, "text": str}

if clean_in.exists():
    data = json.loads(clean_in.read_text(encoding="utf-8"))
    for row in data:
        page = int(row["page"])
        for piece in row.get("pieces", []):
            piece = piece.strip()
            if piece:
                targets.append({"page": page, "text": piece})
else:
    # fallback to raw ids
    ids = json.loads(ids_path.read_text(encoding="utf-8"))
    lookup = json.loads(lookup_path.read_text(encoding="utf-8"))
    for sid in ids:
        ent = lookup.get(sid)
        if ent:
            targets.append({"page": int(ent["page"]), "text": ent["sentence"]})

# group targets per page
by_page = defaultdict(list)
for t in targets:
    by_page[int(t["page"])].append(t["text"])

# ---------- highlight
doc = fitz.open(pdf_in)
total_spans, misses = 0, []

for page_num, pieces in sorted(by_page.items()):
    page = doc[page_num - 1]
    blocks = build_blocks(page)

    for raw in pieces:
        if not raw.strip():
            continue

        found = False
        sentence_rects = None
        target_tokens = norm_tokenize(raw)
        if not target_tokens:
            continue

        # 1) block-scoped exact token window
        for _b_idx, wlist in blocks.items():
            stream, tok2word = stream_from_words(wlist)
            loc = exact_window(stream, target_tokens)
            if loc:
                rlist = rects_from_span(wlist, tok2word, *loc)
                for r in rlist:
                    page.add_highlight_annot(r)
                    total_spans += 1
                sentence_rects = rlist
                found = True
                break

        # 2) page-wide exact token window
        if not found:
            all_words = [w for bl in blocks.values() for w in bl]
            stream, tok2word = stream_from_words(all_words)
            loc = exact_window(stream, target_tokens)
            if loc:
                rlist = rects_from_span(all_words, tok2word, *loc)
                for r in rlist:
                    page.add_highlight_annot(r)
                    total_spans += 1
                sentence_rects = rlist
                found = True

        # 3) search_for with paren/prime variants (no stripping of primes!)
        if not found:
            for base in paren_variants(raw):
                for variant in prime_variants(base):
                    v = unicodedata.normalize("NFKC", variant).replace("\u00ad", "")
                    v = fold_diacritics(v)
                    # normalize hyphens/quotes spacing but DO NOT remove primes
                    v = re.sub(HYPHENS, " ", v)
                    v = re.sub(QUOTES, " ", v)
                    v = re.sub(r"\s+", " ", v).strip()
                    if not v:
                        continue
                    quads = page.search_for(v, quads=True)
                    if quads:
                        for q in quads:
                            page.add_highlight_annot(q)
                            total_spans += 1
                        sentence_rects = [fitz.Rect(q.rect) for q in quads]
                        found = True
                        break
                if found:
                    break

        # 4) tolerant page-wide token window (only for longer phrases)
        if not found and len(target_tokens) >= 10:
            all_words = [w for bl in blocks.values() for w in bl]
            stream, tok2word = stream_from_words(all_words)
            loc = tolerant_window(stream, target_tokens, max_miss=1, min_ratio=0.9)
            if loc:
                rlist = rects_from_span(all_words, tok2word, *loc)
                for r in rlist:
                    page.add_highlight_annot(r)
                    total_spans += 1
                sentence_rects = rlist
                found = True

        # ----- scoped micro-highlights: operators + primes + parentheses -----
        if found and sentence_rects and has_ops_or_primes_or_parens(raw):
            region = union_rect(sentence_rects)
            pad = 1.5
            region = fitz.Rect(region.x0 - pad, region.y0 - pad, region.x1 + pad, region.y1 + pad)

            # single-glyph operators
            for ch in set(OP_CHARS):
                if ch in raw:
                    quads = page.search_for(ch, quads=True)
                    for q in quads:
                        r = fitz.Rect(q.rect)
                        if region.contains(r):
                            page.add_highlight_annot(q)
                            total_spans += 1

            # multi-char ASCII sequences
            for seq in OP_STRINGS:
                if seq in raw:
                    quads = page.search_for(seq, quads=True)
                    for q in quads:
                        r = fitz.Rect(q.rect)
                        if region.contains(r):
                            page.add_highlight_annot(q)
                            total_spans += 1

            # parentheses
            for ch in "()":
                if ch in raw:
                    quads = page.search_for(ch, quads=True)
                    for q in quads:
                        r = fitz.Rect(q.rect)
                        if region.contains(r):
                            page.add_highlight_annot(q)
                            total_spans += 1

            # primes (′ ″ ‴ ⁗)
            for ch in PRIME_SET:
                if ch in raw:
                    quads = page.search_for(ch, quads=True)
                    for q in quads:
                        r = fitz.Rect(q.rect)
                        if region.contains(r):
                            page.add_highlight_annot(q)
                            total_spans += 1

        if not found:
            misses.append(raw[:90])

doc.save(pdf_out, garbage=4, deflate=True)
doc.close()

print(f"✅ Highlighted {total_spans} spans → {pdf_out}")
if misses:
    print("⚠️ Not found (sample):", misses[:6])


✅ Highlighted 263 spans → highlighted_output.pdf


In [218]:
# CELL 5 — robust highlighter (token-aware + tolerant + fully gated operators)
import fitz  # PyMuPDF
import json
import re
import unicodedata
from math import ceil
from pathlib import Path
from collections import defaultdict

# ---------- paths
base_dir = Path(".")
pdf_in   = base_dir / "class10phy.pdf"
pdf_out  = base_dir / "highlighted_output.pdf"
clean_in = base_dir / "ai_cleaned_sentences.json"     # from Cell 4
ids_path = base_dir / "highlight_ids.json"            # fallback if Cell 4 is skipped
lookup_path = base_dir / "sentence_lookup.json"       # fallback

# ---------- normalization / tokenization
HYPHENS = r"[\-\u2010-\u2015\u2212]"  # -, ‐, –, —, −
QUOTES  = r"[\u2018\u2019\u201A\u201C\u201D\u201E']"
PRIME_CHARS = "\u2032\u2033\u2034\u2057"
PRIME_SET = set(PRIME_CHARS)

def fold_diacritics(s: str) -> str:
    s = unicodedata.normalize("NFD", s)
    s = "".join(c for c in s if not unicodedata.combining(c))
    return unicodedata.normalize("NFC", s)

def normalize_for_search(s: str) -> str:
    s = unicodedata.normalize("NFKC", s).replace("\u00ad", "")
    s = fold_diacritics(s)
    s = re.sub(HYPHENS, " ", s)
    s = re.sub(QUOTES, " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def normalize_primes_to_tokens(s: str) -> str:
    return (s.replace("\u2032", " prime")
             .replace("\u2033", " doubleprime")
             .replace("\u2034", " tripleprime")
             .replace("\u2057", " quadrupleprime"))

def prime_variants(s: str):
    yield s
    yield s.translate({ord("\u2032"): ord("'"), ord("\u2033"): ord('"')})
    yield re.sub(f"[{PRIME_CHARS}]", "", s)

def norm_tokenize(s: str):
    s = unicodedata.normalize("NFKC", s).replace("\u00ad", "")
    s = fold_diacritics(s)
    s = normalize_primes_to_tokens(s)
    s = re.sub(HYPHENS, " ", s)
    s = re.sub(QUOTES, " ", s)
    s = re.sub(r"[^\w\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip().lower()
    return s.split() if s else []

def paren_variants(s: str):
    v1 = s
    v2 = re.sub(r"\s*\(\s*", " (", v1); v2 = re.sub(r"\s*\)\s*", ")", v2)
    v3 = re.sub(r"[()]", "", v2); v3 = re.sub(r"\s+", " ", v3).strip()
    return [v1, v2, v3]

# ---------- operator inventory
OPS_MULTI  = ["<=", ">=", "==", "!=", "->", "<-", "<->"]
OPS_SINGLE = list("=+-*/^×·÷:<>≤≥≠→↔∝") + ["−"]  # include Unicode minus

# Optional on-page glyph aliases (keep conservative to avoid false hits)
OP_ALIASES = {
    "-": ["-", "−"],   # ASCII minus & Unicode minus
    "−": ["−", "-"],   # either way
}

EXTRA_REGION_SYMS = "()"

def has_symbols(s: str) -> bool:
    return any(ch in s for ch in OPS_SINGLE) or any(seq in s for seq in OPS_MULTI) or any(ch in s for ch in EXTRA_REGION_SYMS) or any(ch in s for ch in PRIME_SET)

# ---------- PDF helpers
def build_blocks(page):
    words = page.get_text("words")
    if not words:
        return {-1: []}
    has_blocks = len(words[0]) >= 8
    blocks = defaultdict(list)
    if has_blocks:
        for w in words: blocks[w[5]].append(w)
    else:
        blocks[-1] = words
    return blocks

def stream_from_words(wlist):
    stream, tok2word = [], []
    for wi, w in enumerate(wlist):
        toks = norm_tokenize(w[4])
        for t in toks:
            stream.append(t); tok2word.append(wi)
    return stream, tok2word

def rects_from_span(wlist, tok2word, i0, i1):
    idxs = sorted(set(tok2word[i0:i1+1]))
    return [fitz.Rect(*wlist[idx][0:4]) for idx in idxs]

def exact_window(stream, target):
    n, m = len(stream), len(target)
    if m == 0 or n < m: return None
    for i in range(n - m + 1):
        if stream[i:i+m] == target:
            return (i, i+m-1)
    return None

def tolerant_window(stream, target, max_miss=1, min_ratio=0.9):
    n, m = len(stream), len(target)
    if m == 0 or n < m: return None
    need = max(int(ceil(min_ratio * m)), m - max_miss)
    for i in range(n - m + 1):
        win = stream[i:i+m]
        matches = sum(1 for a, b in zip(win, target) if a == b)
        if matches >= need:
            return (i, i+m-1)
    return None

def union_rect(rects):
    if not rects: return None
    u = fitz.Rect(rects[0])
    for r in rects[1:]: u |= r
    return u

def median_height(rects):
    if not rects: return 1.0
    hs = sorted((r.y1 - r.y0) for r in rects)
    k = len(hs)//2
    return hs[k] if len(hs)%2 else 0.5*(hs[k-1]+hs[k])

def same_line(r1, r2, ypad):
    yc = 0.5*(r1.y0+r1.y1)
    return (r2.y0-ypad) <= yc <= (r2.y1+ypad)

# ---------- general gated operators
TOKEN = r"(?:\d+(?:\.\d+)?|[A-Za-z](?:\d+)?)"   # 12, 1.33, v2, n1, x

def pairs_for_op(text: str, op: str):
    """Extract (left_token, right_token) pairs for a specific operator from the piece text."""
    esc = re.escape(op)
    pairs = []
    # Allow optional spaces around operator
    for m in re.finditer(rf"({TOKEN})\s*{esc}\s*({TOKEN})", text):
        pairs.append((m.group(1), m.group(2)))
    return pairs

def search_token(page, region, tok):
    tok = normalize_for_search(tok)
    hits = []
    for q in page.search_for(tok, quads=True):
        r = fitz.Rect(q.rect)
        if region.contains(r):
            hits.append(r)
    return hits

def collect_glyph_quads(page, region, op):
    """All quads for operator glyphs inside region, with alias support."""
    quads = []
    glyphs = OP_ALIASES.get(op, [op])
    for g in glyphs:
        quads.extend([q for q in page.search_for(g, quads=True) if region.contains(fitz.Rect(q.rect))])
    return quads

def score_op_quad(qr, Lr, Rr, ypad):
    if not (same_line(qr, Lr, ypad) and same_line(qr, Rr, ypad)):
        return 1e9
    midx = 0.5*(Lr.x1 + Rr.x0)
    qx   = 0.5*(qr.x0 + qr.x1)
    # prefer centered between tokens and actually between them
    between_pen = 0.0 if (Lr.x1 <= qr.x0 and qr.x1 <= Rr.x0) else 10_000.0
    center_pen  = 0.1 * abs(qx - midx)
    return between_pen + center_pen

def highlight_op_gated(page, region, sentence_rects, piece_text, op, total_spans):
    """Gate a single operator type for this piece. Returns new spans added."""
    added = 0
    # Special-case multi-char ops FIRST (they’re unique sequences)
    pairs = pairs_for_op(piece_text, op)
    need = len(pairs)
    if need == 0:
        return 0

    op_quads = collect_glyph_quads(page, region, op)
    if not op_quads:
        return 0

    ypad = max(1.0, 0.35*median_height(sentence_rects))
    used = set()

    for Lt, Rt in pairs:
        Ls = search_token(page, region, Lt)
        Rs = search_token(page, region, Rt)
        if not Ls or not Rs:
            continue
        # choose best L/R on same baseline with minimal gap
        best_pair = None; best_gap = 1e18
        for Lr in Ls:
            for Rr in Rs:
                if Rr.x0 <= Lr.x1:  # wrong order
                    continue
                if not same_line(Lr, Rr, ypad):
                    continue
                gap = Rr.x0 - Lr.x1
                if gap < best_gap:
                    best_gap = gap; best_pair = (Lr, Rr)
        if not best_pair:
            continue

        Lr, Rr = best_pair
        # pick the op glyph between L and R
        candidates = []
        for i, q in enumerate(op_quads):
            if i in used: continue
            qr = fitz.Rect(q.rect)
            s = score_op_quad(qr, Lr, Rr, ypad)
            if s < 9e8:
                candidates.append((s, i))
        if candidates:
            candidates.sort(key=lambda t: t[0])
            idx = candidates[0][1]
            page.add_highlight_annot(op_quads[idx])
            used.add(idx)
            added += 1

    # Fallback: if anchors failed (kerning/OCR), select remaining best matches on the midline
    if added < need:
        yc = sorted((0.5*(r.y0+r.y1) for r in sentence_rects))[len(sentence_rects)//2]
        residual = []
        for i, q in enumerate(op_quads):
            if i in used: continue
            qr = fitz.Rect(q.rect)
            residual.append((abs(0.5*(qr.y0+qr.y1)-yc), qr.x0, i))
        residual.sort()
        for _, _, i in residual[:max(0, need-added)]:
            page.add_highlight_annot(op_quads[i])
            used.add(i)
            added += 1

    return added

# ---------- load targets
targets = []
if clean_in.exists():
    data = json.loads(clean_in.read_text(encoding="utf-8"))
    for row in data:
        page = int(row["page"])
        for piece in row.get("pieces", []):
            piece = piece.strip()
            if piece:
                targets.append({"page": page, "text": piece})
else:
    ids = json.loads(ids_path.read_text(encoding="utf-8"))
    lookup = json.loads(lookup_path.read_text(encoding="utf-8"))
    for sid in ids:
        ent = lookup.get(sid)
        if ent:
            targets.append({"page": int(ent["page"]), "text": ent["sentence"]})

by_page = defaultdict(list)
for t in targets:
    by_page[int(t["page"])].append(t["text"])

# ---------- highlight
doc = fitz.open(pdf_in)
total_spans, misses = 0, []

for page_num, pieces in sorted(by_page.items()):
    page = doc[page_num - 1]
    blocks = build_blocks(page)

    for raw in pieces:
        if not raw.strip():
            continue

        found = False
        sentence_rects = None
        target_tokens = norm_tokenize(raw)
        if not target_tokens:
            continue

        # 1) block-scoped exact token window
        for _b_idx, wlist in blocks.items():
            stream, tok2word = stream_from_words(wlist)
            loc = exact_window(stream, target_tokens)
            if loc:
                rlist = rects_from_span(wlist, tok2word, *loc)
                for r in rlist:
                    page.add_highlight_annot(r); total_spans += 1
                sentence_rects = rlist; found = True
                break

        # 2) page-wide exact token window
        if not found:
            all_words = [w for bl in blocks.values() for w in bl]
            stream, tok2word = stream_from_words(all_words)
            loc = exact_window(stream, target_tokens)
            if loc:
                rlist = rects_from_span(all_words, tok2word, *loc)
                for r in rlist:
                    page.add_highlight_annot(r); total_spans += 1
                sentence_rects = rlist; found = True

        # 3) search_for with paren/prime variants
        if not found:
            for base in paren_variants(raw):
                for variant in prime_variants(base):
                    v = normalize_for_search(variant)
                    if not v: continue
                    quads = page.search_for(v, quads=True)
                    if quads:
                        for q in quads:
                            page.add_highlight_annot(q); total_spans += 1
                        sentence_rects = [fitz.Rect(q.rect) for q in quads]
                        found = True; break
                if found: break

        # 4) tolerant page-wide token window
        if not found and len(target_tokens) >= 10:
            all_words = [w for bl in blocks.values() for w in bl]
            stream, tok2word = stream_from_words(all_words)
            loc = tolerant_window(stream, target_tokens, max_miss=1, min_ratio=0.9)
            if loc:
                rlist = rects_from_span(all_words, tok2word, *loc)
                for r in rlist:
                    page.add_highlight_annot(r); total_spans += 1
                sentence_rects = rlist; found = True

        if not found:
            misses.append(raw[:90])
            continue

        # Region for this sentence (tight, tiny pad)
        region = union_rect(sentence_rects)
        pad = 1.5
        region = fitz.Rect(region.x0 - pad, region.y0 - pad, region.x1 + pad, region.y1 + pad)

        # ----- gated operators for ALL ops -----
        # Multi-char ops first to avoid double counting (e.g., '<=' vs '<' and '=')
        for op in OPS_MULTI:
            total_spans += highlight_op_gated(page, region, sentence_rects, raw, op, total_spans)
        for op in OPS_SINGLE:
            total_spans += highlight_op_gated(page, region, sentence_rects, raw, op, total_spans)

        # ----- micro: parentheses & primes (kept)
        if any(ch in raw for ch in "()"):
            for ch in "()":
                for q in page.search_for(ch, quads=True):
                    r = fitz.Rect(q.rect)
                    if region.contains(r):
                        page.add_highlight_annot(q); total_spans += 1
        for ch in PRIME_SET:
            if ch in raw:
                for q in page.search_for(ch, quads=True):
                    r = fitz.Rect(q.rect)
                    if region.contains(r):
                        page.add_highlight_annot(q); total_spans += 1

doc.save(pdf_out, garbage=4, deflate=True)
doc.close()

print(f"✅ Highlighted {total_spans} spans → {pdf_out}")
if misses:
    print("⚠️ Not found (sample):", misses[:6])


✅ Highlighted 261 spans → highlighted_output.pdf
